# 1-Cocycles for n-braids

## I - Main functions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import warnings
warnings.filterwarnings("ignore")
from numpy import *
from copy import *

Define the number of brands for the braid

In [ ]:
Nbraid = 4
B = BraidGroup(Nbraid)

Generating all possible types

In [ ]:
def generate_all_possible_global_types(Nbraid):
    '''A function that generates all possible global types for a triple point of a Nbraid-braid'''
    ListType=[]
    #Negative type
    for ml in range(1,Nbraid-1):
        for hm in range(1,Nbraid-ml):
            ListType.append([ml,hm,'-'])
    #Positive type
    for d in range(1,Nbraid-1):
        for ml in range(d+1,Nbraid):
            hm = d + Nbraid - ml
            ListType.append([ml,hm,'+'])
    return ListType

In [ ]:
ListType = generate_all_possible_global_types(Nbraid)
print(ListType)

### I.a. Translating n-braids into BraidData
Functions to represent closed braids in the solid torus

In [ ]:
#Function to represent closed braids
def plot_one_strand(x,y,color='k',intercross = 0.5):
    '''A function that plots a straight strand at coordinate (x,y) of length 0.5 '''
    abscisse = linspace(x,x+intercross,50)
    ordonne = zeros(50)
    plt.plot(abscisse,ordonne + y, color)

def plot_intercrossing(x,y,N,color='k',intercross=0.5): 
    '''A function that plots the intercrossing of an N-braid with strand 1 at coordinate (x,y) of length 0.5'''
    for i in range(0,N):
        plot_one_strand(x,y+i,color,intercross)
        
def plot_local_crossing(x,i,sign,size=0.05,color='k',length=1):
    '''A function that plot locally a crossing at coordinate (x,i)'''
    if sign < 0 :
        #Overcrossing
        a = linspace(x,x+length,50)
        b = a/length + (i-(x/length))
        plt.plot(a,b,color)
        #Undercrossing
        xmil = x + (length/2)
        a = linspace(x,xmil-size,50)
        b = -a/length + i + 1 + (x/length)
        plt.plot(a,b,color)
        xmil = x + (length/2)
        a = linspace(xmil+size,x+length,50)
        b = -a/length + i + 1 + (x/length)
        plt.plot(a,b,color)
    elif sign > 0 :
        #Overcrossing
        a = linspace(x,x+length,50)
        b = -a/length + i + 1 + (x/length)
        plt.plot(a,b,color)
        #Undercrossing
        xmil = x + (length/2)
        a = linspace(x,xmil-size,50)
        b = a/length + (i-(x/length))
        plt.plot(a,b,color)
        xmil = x + (length/2)
        a = linspace(xmil+size,x+length,50)
        b = a/length + (i-(x/length))
        plt.plot(a,b,color)

def plot_crossing(x,sigma,N,size=0.05,color='k',length=1):
    '''A function that plot the crossing sigma at coordinate x of an N-braid''' 
    plot_local_crossing(x,abs(sigma),sigma,size,color,length)
    for i in range(1,N+1):
        if i != abs(sigma) and i!= abs(sigma)+1 :
            plot_one_strand(x,i,color,length)

def plot_half_circle(center,r=0.5,sign=-1,color='k'):
    '''A function that plot half a circle'''
    if sign > 0:
        theta = linspace(-pi/2,pi/2,50)
    elif sign <0:
        theta = linspace(pi/2,3*pi/2,50)
    abscisse = r*cos(theta) + center[0]
    ordonne = r*sin(theta) + center[1]
    plt.plot(abscisse,ordonne,color)
    
    
def plot_border(x,y,N,sign,color='k'):
    '''A function that plots the border of a N-braid at coordinates (x,y)'''
    center = (x,y)
    for i in range(1,N+1):
        plot_half_circle(center,i,sign,color)
    
def plot_braid(S,color='k',intercross=0.5,size=0.05,length=1,res=300):
    '''A function that plot a braid S'''
    fig = plt.figure(dpi=res)
    N = max(vectorize(abs)(S)) + 1
    #Plotting the crossings
    plot_intercrossing(0,1,N,color,intercross)
    x = intercross
    for sigma in S:
        plot_crossing(x,sigma,N,size,color,length)
        x = x + length
        plot_intercrossing(x,1,N,color,intercross)
        x = x + intercross
    #Plotting the "lower" strands below the center hole
    plot_intercrossing(0,-N,N,color,len(S)*(intercross+length)+intercross)
    #Plotting the turns
    plot_border(0,0,N,-1,color)
    x = len(S)*(intercross+length)+intercross
    y = 0
    plot_border(x,y,N,1,color)
    #Plotting the singular point
    x = (len(S)*(intercross+length)+intercross)/2
    motif = color + 'o'
    plt.plot(x,0,motif)
    plt.ylim([-N-0.5,N+0.5 ])
    plt.axis('off')
    plt.axis('scaled')

Functions to generate BraidData and rot

In [ ]:
def end_homotopy_class(braid):
    L = Link(B(braid))
    #Getting the last permutation of braid
    n = len(braid)
    p = braid[n-1]
    h = 0
    #Computing the homotopy class of the last crossing
    if abs(p) == 1 :
        if L.braid().permutation().cycle_string() == '(1,2,3,4)':
            h = 1
        if L.braid().permutation().cycle_string() == '(1,2,4,3)':
            h = 1
        if L.braid().permutation().cycle_string() == '(1,3,2,4)':
            h = 2
        if L.braid().permutation().cycle_string() == '(1,3,4,2)':
            h = 3
        if L.braid().permutation().cycle_string() == '(1,4,2,3)':
            h = 2
        if L.braid().permutation().cycle_string() == '(1,4,3,2)':
            h = 3

    if abs(p) == 2 :
        if L.braid().permutation().cycle_string() == '(1,2,3,4)':
            h = 1
        if L.braid().permutation().cycle_string() == '(1,2,4,3)':
            h = 2
        if L.braid().permutation().cycle_string() == '(1,3,2,4)':
            h = 3
        if L.braid().permutation().cycle_string() == '(1,3,4,2)':
            h = 2
        if L.braid().permutation().cycle_string() == '(1,4,2,3)':
            h = 1
        if L.braid().permutation().cycle_string() == '(1,4,3,2)':
            h = 3
            
    if abs(p) == 3 :
        if L.braid().permutation().cycle_string() == '(1,2,3,4)':
            h = 1
        if L.braid().permutation().cycle_string() == '(1,2,4,3)':
            h = 3
        if L.braid().permutation().cycle_string() == '(1,3,2,4)':
            h = 2
        if L.braid().permutation().cycle_string() == '(1,3,4,2)':
            h = 1
        if L.braid().permutation().cycle_string() == '(1,4,2,3)':
            h = 2
        if L.braid().permutation().cycle_string() == '(1,4,3,2)':
            h = 3
            
    #If the permutation is negative, just take 4-h
    if sign(p) == -1 :
        h = 4-h
        
    return h

def any_homotopy_class(braid,i):
    T = deepcopy(braid)
    #Making braid[i] the last permutation
    for j in range(i+1):
        T.append(T[0])
        T.pop(0)
    h = end_homotopy_class(T)
    return h

def gauss_coordinates(GD,i):
    '''A function that computes the coordinates of the arrow number i of a torus gauss code'''
    #In : GD = GaussDiagram as a list, i number of the arrow 
    #Out : C a list of 2 coordinates

    n = len(GD)
    C = ['empty','empty']
    #Getting the first coordinate of the arrow i
    for j in range(n):
        if GD[j] == i:
            C[0] = j
    #Getting the second coordinate of the arrow i
        if GD[j] == -i:
            C[1] = j
    return C

def TorusGaussCode_reconstruct(A):
    '''A function that computes a TorusGaussCode from a set of TorusGaussArrows A'''
    T = TorusGaussCode()
    #Setting the arrows
    T.set_Arrows(A)
    #Computing the Gauss Code from the arrows coordinates
    l = len(A)
    GD = list(zeros(2*l))
    #For each arrow, we modify the gauss diagram
    for a in A:
        C = a.get_coordinates()
        n = a.get_number()
        #Foot first
        GD[C[0]] = n
        #Then head
        GD[C[1]] = -n
    T.set_GaussDiagram(GD)
    return T

def compute_arrow(braid,i,number):
    '''A function that computes the TorusGaussArrow of a braid in position i'''
    A = TorusGaussArrow()
    L = Link(B(braid))
    #Arrow number
    A.set_number(number)
    #Arrow position
    A.set_position(i)
    #Arrow braid element
    A.set_braidelement(braid[i])
    #Arrow homotopy
    A.set_homotopy(any_homotopy_class(braid,i))
    #Arrow writhe
    A.set_writhe(L.oriented_gauss_code()[1][i])
    GD = L.oriented_gauss_code()[0][0]
    #Arrow coordinates
    A.set_coordinates(gauss_coordinates(GD,number))
    return A

#Function for the RM2
def coordinates_RM2(B,e,i):
    '''A function that returns the coordinates of the two arrows created in a RM2 in position i with braid element e'''
    #Initialisation
    braid = B.get_BraidClosure()
    A = B.get_TorusGaussCode().get_Arrows()
    m = len(braid) - 1
    E = abs(e)
    #Computing the coordinates
    f1 = 'INIT'
    h1 = 'INIT'
    for j in range(m):
        if abs(braid[i-j-1]) == E:
            X = A[i-j-1]
            Xf = X.get_coordinates()[0]
            Xh = X.get_coordinates()[1]
            if braid[i-j-1] < 0:
                Xf,Xh = Xh,Xf
            if type(f1) == str:
                f1 = (Xh + 1)%(2*m)
                position_element_foot = i-j-1
            if type(h1) == str:
                h1 = (Xf + 1)%(2*m)
                position_element_head = i-j-1
        if abs(braid[i-j-1]) == E-1:
            X = A[i-j-1]
            Xf = X.get_coordinates()[0]
            Xh = X.get_coordinates()[1]
            if braid[i-j-1] < 0:
                Xf,Xh = Xh,Xf
            if type(f1) == str:
                f1 = (Xf + 1)%(2*m)
                position_element_foot = i-j-1
        if abs(braid[i-j-1]) == E+1:
            X = A[i-j-1]
            Xf = X.get_coordinates()[0]
            Xh = X.get_coordinates()[1]
            if braid[i-j-1] < 0:
                Xf,Xh = Xh,Xf
            if type(h1) == str:
                h1 = (Xh + 1)%(2*m)
                position_element_head = i-j-1
            
        if type(f1) != str and type(h1) != str:
            break
    #Adjusting for negative first element
    if e < 0 :
        f1,h1 = h1,f1
    #Computing coordinates for the second arrow
    f2 = (f1 + 1)%(2*m)
    h2 = (h1 + 1)%(2*m)
    
    #Particular Case
    if position_element_foot < 0:
        f1 = (f1 + 2)%(2*m)
        f2 = (f2 + 2)%(2*m)
    elif position_element_head < 0:
        h1 = (h1 + 2)%(2*m)
        h2 = (h2 + 2)%(2*m)
    #Adjusting the two greater coordinates
    if f1 < h1 :
        h1 = h1 + 2
        h2 = h2 + 2
    else:
        f1 = f1 + 2
        f2 = f2 + 2
    return [f1,h1],[f2,h2]

#Functions for the RM3
def find_arrow_coord(coord,A):
    '''A function that finds the arrow in a set A that have coordinates coord'''
    arrow = TorusGaussArrow()
    for a in A:
        if a.get_coordinates()[0] == coord or a.get_coordinates()[1] == coord :
            arrow = a
    return arrow

def find_arrow_position(A,i):
    '''A function that returns the arrow that is in position i'''
    ans = TorusGaussArrow()
    for a in A:
        if a.get_position() == i:
            ans = a
    return ans

def find_arrow_number(A,i):
    '''A function that returns the arrow that is in position i'''
    ans = TorusGaussArrow()
    for a in A:
        if a.get_number() == i:
            ans = a
    return ans

def find_switchables(coord,L,numb):
    '''A function that returns all the switchable coordinates from a set of coordinates L'''
    Switchables = []
    for c in L:
        if (c-coord)%numb == 1 or (coord-c)%numb == 1:
            Switchables.append(c)
    return Switchables

def is_switchable_once(coord,coord1,L,numb):
    '''A function that determines in a list if a coordinate is switchable with only one coordinate (coord1 is a forbidden switchable coordinate)'''
    Switchables = find_switchables(coord,L,numb)
    if coord1 in Switchables:
        Switchables.remove(coord1)
    if len(Switchables) == 1:
        return True
    else:
        return False

def switch_coordinates(coord1,Switchables,A):
    '''A function that switches two coordinates in a set of arrows'''
    if len(Switchables) != 1:
        print('ERROR : too many coordinates to choose from')
        return False
    else:
        coord2 = Switchables[0]
        a1 = find_arrow_coord(coord1,A)
        a2 = find_arrow_coord(coord2,A)
        i1 = a1.get_position()
        i2 = a2.get_position()
        C1 = a1.get_coordinates()
        C2 = a2.get_coordinates()
        for i in range(2):
            if C1[i] == coord1:
                C1[i] = coord2
            if C2[i] == coord2:
                C2[i] = coord1
        a1.set_coordinates(C1)
        a2.set_coordinates(C2)
        A[i1] = a1
        A[i2] = a2
        return True
    
def compute_coordinates(coord1,Switchables,A):
    '''A function that switches two coordinates in a set of arrows'''
    if len(Switchables) != 1:
        print('ERROR : too many coordinates to choose from')
        return False
    else:
        coord2 = Switchables[0]
        a1 = find_arrow_coord(coord1,A)
        a2 = find_arrow_coord(coord2,A)
        i1 = a1.get_position()
        i2 = a2.get_position()
        C1 = a1.get_coordinates()
        C2 = a2.get_coordinates()
        coord = min(coord1,coord2)
        for i in range(2):
            if C1[i] == coord1:
                C1[i] = coord
            if C2[i] == coord2:
                C2[i] = coord
        a1.set_coordinates(C1)
        a2.set_coordinates(C2)
        A[i1] = a1
        A[i2] = a2
        return True
    
def same_foot(A,a):
    '''A function that computes if the arrow a has same foot coordinates as another arrow in A'''
    ans = False
    for arrow in A:
        if arrow != a and a.get_coordinates()[0] == arrow.get_coordinates()[0]:
            ans = True
    return ans

def same_head(A,a):
    '''A function that computes if the arrow a has same head coordinates as another arrow in A'''
    ans = False
    for arrow in A:
        if arrow != a and a.get_coordinates()[1] == arrow.get_coordinates()[1]:
            ans = True
    return ans


def name_arrows(a,b,c):
    '''A function that returns the name of three singular arrows in a singular RM3 diagram'''
    A = [a,b,c]
    for arrow in A:
        if same_head(A,arrow):
            if same_foot(A,arrow):
                d = arrow
            else:
                hm = arrow
        else:
            ml = arrow
    return d,hm,ml
    
def local_type(D):
    '''A function that returns the local type of a singular BraidData'''
    D1 = deepcopy(D)
    #First, we check that we are in a singular case
    if D1.get_Class() == 'regular':
        print('Erreur : ce n\' est pas un diagramme singulier')
    else:
        clas = D1.get_Class()
        a,b,c = D1.get_RM3_a(),D1.get_RM3_b(),D1.get_RM3_c()
        ba = a.get_braidelement()
        bb = b.get_braidelement()
        bc = c.get_braidelement()
        rm3 = (ba,bb,bc)
        lt = -1
        n = 4
        for i in range(1,n):
            if rm3 == (i+1,i,i+1) or rm3 == (-(i+1),-i,i+1) or rm3 == (i+1,-i,-(i+1)) or rm3 == (-i,i+1,i) or rm3 == (i,i+1,-i) or rm3 == (-i,-(i+1),-i):
                lt = 1
        return lt

def global_type(D):
    '''A function that returns the global type of a singular BraidData'''
    D1 = deepcopy(D)
    #First, we check that we are in a singular case
    if D1.get_Class() == 'regular':
        print('Erreur : ce n\' est pas un diagramme singulier')
    else:
        clas = D1.get_Class()
        a,b,c = D1.get_RM3_a(),D1.get_RM3_b(),D1.get_RM3_c()
        d,hm,ml = name_arrows(a,b,c)
        homd = d.get_homotopy()
        homhm = hm.get_homotopy()
        homml = ml.get_homotopy()
        if homd == homhm + homml:
            gt = -1
        else:
            gt = 1
    return gt

Classes for BraidData

In [ ]:
class TorusGaussArrow:
    def __init__(self):
        self.__number = 'INIT'
        self.__position = 'INIT'
        self.__braidelement = 'INIT'
        self.__homotopy = 0
        self.__writhe = 0
        self.__coordinates = 'empty'
        self.__class = 'regular'
    def set_number(self,num):
        self.__number = num
    def set_position(self,pos):
        self.__position = pos
    def set_braidelement(self,t):
        self.__braidelement = t
    def set_homotopy(self,h):
        self.__homotopy = h
    def set_writhe(self,w):
        self.__writhe = w
    def set_coordinates(self,c):
        self.__coordinates = c
    def set_class(self,c):
        self.__class = c
    def get_number(self):
        return self.__number
    def get_position(self):
        return self.__position
    def get_braidelement(self):
        return self.__braidelement
    def get_homotopy(self):
        return self.__homotopy
    def get_writhe(self):
        return self.__writhe
    def get_coordinates(self):
        return self.__coordinates
    def get_class(self):
        return self.__class
    def get_infos(self):
        print('Number : ',self.get_number())
        print('Position in braid : ',self.get_position())
        print('Braid element : ',self.get_braidelement())
        print('Writhe : ',self.get_writhe())
        print('Homotopy : ',self.get_homotopy())
        print('Coordinates : ',self.get_coordinates() )
    def __eq__(self,A):
        if A.__class__.__name__ != "TorusGaussArrow" :
            print('Error : Could not test the operand ==. Unsupported type for the second argument')
            return False
        else:
            if self.get_number() == A.get_number() and self.get_writhe() == A.get_writhe() and self.get_homotopy() == A.get_homotopy() and self.get_coordinates() == A.get_coordinates() :
                return True
            else:
                return False
        
class TorusGaussCode :
    "A Gauss Diagram of a braid in the solid torus"
    def __init__(self,braid=0):
        self.__Arrows = []
        if braid == 0:
            self.__GaussDiagram = []
        else:
            L = Link(B(braid))
            self.__GaussDiagram = L.oriented_gauss_code()[0][0]
            for i in range(len(braid)):
                A = compute_arrow(braid,i,i+1)
                self.__Arrows.append(A)
    def set_GaussDiagram(self,G):
        self.__GaussDiagram = G
    def set_Arrows(self,A):
        self.__Arrows = A
    def get_GaussDiagram(self):
        return self.__GaussDiagram
    def get_Arrows(self):
        return self.__Arrows
    def get_infos(self):
        print('Gauss Diagram : ',self.get_GaussDiagram())
    def remove_arrow(self,i):
        #Removing the arrow in position i
        A = self.get_Arrows()
        for arrow in A:
            if arrow.get_position() == i:
                self.get_Arrows().remove(arrow)
                #Removing from the GaussDiagram
                j = arrow.get_number()
                self.get_GaussDiagram().remove(j)
                self.get_GaussDiagram().remove(-j)
    def plot_diagram(self):
        '''A method that plots the Torus Gauss diagram associated to the TorusGaussCode'''
        #Plotting the circle
        ax = plt.axes()
        radius = 1
        theta = linspace(0, 2*pi, 100)
        x1 = radius*cos(theta)
        x2 = radius*sin(theta)
        plt.plot(x1,x2,'black')
        plt.axis('scaled')
        plt.axis('off')
        #Plotting the base point
        plt.text(0, 0.92,'*',color='black',size = 20,horizontalalignment='center')
        #Plotting the arrows
        n = 2*len(self.get_Arrows())+1
        for a in self.get_Arrows():
            if a.get_homotopy() == 1:
                color = 'red'
            elif a.get_homotopy() == 2:
                color = 'blue'
            elif a.get_homotopy() == 3:
                color = 'green'
            else:
                color = 'black'
            number = a.get_number()
            if a.get_writhe() > 0:
                sign = '+'
            else:
                sign = '-'
            c1,c2 = a.get_coordinates()
            fangle = (c1+1)*2*pi/n + pi/2
            hangle = (c2+1)*2*pi/n + pi/2
            plt.annotate('', xy = (cos(hangle), sin(hangle)) , xytext = (cos(fangle), sin(fangle)),
                            arrowprops = {'color': color,'arrowstyle' : '->'})
            #Labels
            r = 1.15
            plt.text(r*cos(hangle), r*sin(hangle),sign,color=color)
            plt.text(r*cos(fangle), r*sin(fangle),number,color=color)
    def plot_singular_diagram(self):
        '''A method that plots the Torus Gauss diagram associated to the TorusGaussCode'''
        #Plotting the circle
        ax = plt.axes()
        radius = 1
        theta = linspace(0, 2*pi, 100)
        x1 = radius*cos(theta)
        x2 = radius*sin(theta)
        plt.plot(x1,x2,'black')
        plt.axis('scaled')
        plt.axis('off')
        #Plotting the base point
        plt.text(0, 0.92,'*',color='black',size = 20,horizontalalignment='center')
        #Plotting the arrows
        n = 2*len(self.get_Arrows())-2
        for a in self.get_Arrows():
            if a.get_homotopy() == 1:
                color = 'red'
            elif a.get_homotopy() == 2:
                color = 'blue'
            elif a.get_homotopy() == 3:
                color = 'green'
            else:
                color = 'black'
            number = a.get_number()
            if a.get_writhe() > 0:
                sign = '+'
            else:
                sign = '-'
            c1,c2 = a.get_coordinates()
            fangle = (c1+1)*2*pi/n + pi/2
            hangle = (c2+1)*2*pi/n + pi/2
            #Labels
            r = 1.15
            plt.text(r*cos(hangle), r*sin(hangle),sign,color=color)
            if a.get_class() == 'regular':
                plt.text(r*cos(fangle), r*sin(fangle),number,color=color)
                plt.annotate('', xy = (cos(hangle), sin(hangle)) , xytext = (cos(fangle), sin(fangle)),
                            arrowprops = {'color': color,'arrowstyle' : '->'})
            else:
                alpha = (max(fangle,hangle)-min(fangle,hangle))/2
                nangle = pi/2 + hangle - alpha
                m = len(self.get_Arrows())
                nr = 0.05*log(m)
                nx = r*cos(hangle)+nr*cos(nangle)
                ny = r*sin(hangle)+nr*sin(nangle)
                plt.text(nx,ny,number,color=color,weight='bold')
                plt.annotate('', xy = (cos(hangle), sin(hangle)) , xytext = (cos(fangle), sin(fangle)),
                            arrowprops = {'color': color,'width' : 0.7,'headwidth':5,'headlength':5})
                
    def __eq__(self,G):
        if G.__class__.__name__ != "TorusGaussCode" :
            print('Error : Could not test the operand ==. Unsupported type for the second argument')
            return False
        else:
            if self.get_GaussDiagram() == G.get_GaussDiagram() and self.get_Arrows() == G.get_Arrows():
                return True
            else:
                return False
            
class BraidData :
    "The datas of a braid, containing the braid closure, the arrows numbers and the torus gauss code"
    def __init__(self,braid=0):
        if braid == 0:
            self.BraidClosure = []
            self.ArrowsNumber = []
            self.TorusGaussCode = TorusGaussCode()
            self.length = 0
            #To treat singular diagrams
            self.sign = 1
            self.Class = 'regular'
            self.RM3_a = TorusGaussArrow()
            self.RM3_b = TorusGaussArrow()
            self.RM3_c = TorusGaussArrow()
        else:
            L = Link(B(braid))
            if L.is_knot():
                self.TorusGaussCode = TorusGaussCode(braid)
                self.update_infos()
                self.Class = 'regular'
            else:
                print('Error : the braid is not a knot')
    def update_infos(self):
        '''A method that updates the BraidClosure and Numberlist from the arrows'''
        #Computing the BraidClosure and the Numberlist from the Arrows
        braidclosure = list(range(len(self.get_TorusGaussCode().get_Arrows())))
        numlist = list(range(len(self.get_TorusGaussCode().get_Arrows())))
        for a in self.get_TorusGaussCode().get_Arrows():
            e = a.get_braidelement()
            n = a.get_number()
            i = a.get_position()
            braidclosure[i] = e
            numlist[i] = n
        self.set_BraidClosure(braidclosure)
        self.set_ArrowsNumber(numlist)
    def update_arrows(self):
        '''A method that updates the arrows from the Numberlist'''
        numlist = self.get_ArrowsNumber()
        for a in self.get_TorusGaussCode().get_Arrows():
            for i in range(len(numlist)):
                if a.get_number() == numlist[i]:
                    a.set_position(i)
    def update_diagram(self):
        '''A method that updates the Gauss Diagram from the set of arrows'''
        A = self.get_TorusGaussCode().get_Arrows()
        GD = TorusGaussCode_reconstruct(A)
        self.set_TorusGaussCode(GD)
    def set_BraidClosure(self,braid):
        self.BraidClosure = braid
        self.length = len(self.BraidClosure)
    def set_sign(self,s):
        self.sign = s
    def set_Class(self,clas):
        self.Class = clas
    def set_RM3_a(self,a):
        self.RM3_a = a
    def set_RM3_b(self,b):
        self.RM3_b = b
    def set_RM3_c(self,c):
        self.RM3_c = c
    def set_ArrowsNumber(self,num):
        self.ArrowsNumber = num
        self.length = len(self.BraidClosure)
    def set_TorusGaussCode(self,T):
        self.TorusGaussCode = T
    def get_BraidClosure(self):
        return self.BraidClosure
    def get_sign(self):
        return self.sign
    def get_Class(self):
        return self.Class
    def get_RM3_a(self):
        return self.RM3_a
    def get_RM3_b(self):
        return self.RM3_b
    def get_RM3_c(self):
        return self.RM3_c
    def get_ArrowsNumber(self):
        return self.ArrowsNumber
    def get_TorusGaussCode(self):
        return self.TorusGaussCode
    def get_length(self):
        self.length = len(self.BraidClosure)
        return self.length
    def get_infos(self):
        self.update_infos()
        print('Braid length : ',self.get_length())
        print('Braid closure : ',self.get_BraidClosure())
        print('Arrows number : ',self.get_ArrowsNumber())
        self.TorusGaussCode.get_infos()
    def remove_arrow(self,i):
        '''A method that removes the arrow in position i'''
        #Deleting arrow in position i
        self.get_TorusGaussCode().remove_arrow(i)
        self.get_BraidClosure().pop(i)
        self.get_ArrowsNumber().pop(i)
        self.update_arrows()
        self.update_infos()
    def plot_diagram(self):
        '''A method that plot the Torus Gauss diagram'''
        if self.get_Class() == 'regular':
            self.update_diagram()
            self.get_TorusGaussCode().plot_diagram()
            braid = self.get_BraidClosure()
            braidtitle = 'Braid :  %s'%braid
            plt.text(0,1.5,braidtitle,color='black',horizontalalignment='center')
            arrowsnum = self.get_ArrowsNumber()
            arrowsnumtitle = 'Arrows : %s'%arrowsnum
            plt.text(0,1.3,arrowsnumtitle,color='black',horizontalalignment='center')
        else:
            self.update_diagram()
            self.get_TorusGaussCode().plot_singular_diagram()
            move = self.get_Class()
            plt.text(0,1.7,move,color='black',weight='bold',horizontalalignment='center')
            braid = self.get_BraidClosure()
            braidtitle = 'Braid :  %s'%braid
            plt.text(0,1.5,braidtitle,color='black',weight='bold',horizontalalignment='center')
            arrowsnum = self.get_ArrowsNumber()
            arrowsnumtitle = 'Arrows : %s'%arrowsnum
            plt.text(0,1.3,arrowsnumtitle,color='black',weight='bold',horizontalalignment='center')
    def CommutationMove(self,i):
        ''' A method that commutes the arrows in position i and i+1 (Commutation Move)'''
        #Testing if the move is allowed
        A = self.get_TorusGaussCode().get_Arrows()
        a = find_arrow_position(A,i).get_braidelement()
        b = find_arrow_position(A,i+1).get_braidelement()
        if abs(abs(a)-abs(b)) > 1 :
            BC = self.get_BraidClosure()
            BC[i],BC[i+1] = BC[i+1],BC[i]
            self.set_BraidClosure(BC)
            AN = self.get_ArrowsNumber()
            AN[i],AN[i+1] = AN[i+1],AN[i]
            self.set_ArrowsNumber(AN)
            A[i],A[i+1] = A[i+1],A[i]
            self.update_arrows()
            self.update_infos()
        else:
            print('ERROR : the braid elements don\'t correspond to a CM')
            print('Position of arrows : ',i,i+1)
            print('Braid elements : ',a,b)
    def ReidemeisterMoveII_delete(self,i):
        '''A method that removes the arrows in position i and i+1 (Reidemeister Move II)'''
        length = self.get_length()
        j = i+1
        #Checking for particular case
        if i == length-1:
            j = 0
        for a in self.get_TorusGaussCode().get_Arrows():
            n1 = self.get_ArrowsNumber()[i]
            if a.get_position() == i:
                e1 = a.get_braidelement()
                n1 = a.get_number()
                f1,h1 = a.get_coordinates()
            if a.get_position() == j:
                e2 = a.get_braidelement()
                n2 = a.get_number()
                f2,h2 = a.get_coordinates()
        #Testing if the braid elements are eligible to a RM2
        if e1 == -e2 :
            self.remove_arrow(i)
            if i == length-1:
                self.remove_arrow(0)
            else:
                self.remove_arrow(i)
            #Updating the coordinates for the other arrows
            #Particular case 1
            if min(f1,f2) == 0 and max(f1,f2) == len(self.get_TorusGaussCode().get_GaussDiagram())+3:
                for a in self.get_TorusGaussCode().get_Arrows():              
                    C = a.get_coordinates()
                    if C[0] < min(h1,h2):
                        C[0] = C[0] - 1
                    if C[0] > max(h1,h2):
                        C[0] = C[0] - 3
                    if C[1] < min(h1,h2):
                        C[1] = C[1] - 1
                    if C[1] > max(h1,h2):
                        C[1] = C[1] - 3
                self.update_arrows()
            #Particular case 2
            elif min(h1,h2) == 0 and max(h1,h2) == len(self.get_TorusGaussCode().get_GaussDiagram())+3:
                for a in self.get_TorusGaussCode().get_Arrows():              
                    C = a.get_coordinates()
                    if C[0] < min(f1,f2):
                        C[0] = C[0] - 1
                    if C[0] > max(f1,f2):
                        C[0] = C[0] - 3
                    if C[1] < min(f1,f2):
                        C[1] = C[1] - 1
                    if C[1] > max(f1,f2):
                        C[1] = C[1] - 3
                self.update_arrows()                                         
            #Regular case
            else:
                for a in self.get_TorusGaussCode().get_Arrows():
                    C = a.get_coordinates()
                    if C[0] > min(max(f1,f2),max(h1,h2)) and C[0] < max(min(f1,f2),min(h1,h2)):
                        C[0] = C[0] - 2
                    if C[1] > min(max(f1,f2),max(h1,h2)) and C[1] < max(min(f1,f2),min(h1,h2)):
                        C[1] = C[1] - 2
                    if C[0] > max(f1,h1,h2,f2):
                        C[0] = C[0] - 4
                    if C[1] > max(f1,h1,h2,f2):
                        C[1] = C[1] - 4
                self.update_arrows()
        else:
            print('ERROR : the braid elements don\'t correspond to a RM2')
            print('Position of the arrows')
            print(i,i+1)
            print('Braid elements')
            print(e1,e2)
        return deepcopy(self)
    def ReidemeisterMoveII_create(self,e,i):
        '''A method that adds the arrows corresponding to the braid elements e,-e in position i and i+1 (Reidemeister Move II)'''
        #Updating the BraidClosure and the ArrowsNumber
        self.get_BraidClosure().insert(i,-e)
        self.get_BraidClosure().insert(i,e)
        m = max(self.get_ArrowsNumber())
        self.get_ArrowsNumber().insert(i,m+2)
        self.get_ArrowsNumber().insert(i,m+1)
        #We can't compute the arrows from the BraidClosure as an RM2 will automatically be cancelled in the Gauss diagram
        #We have to compute them manually
        braid = self.get_BraidClosure()
        s = sign(e)
#         print(braid)
        h = any_homotopy_class(braid,i)
        #Computations for the coordinates
        C1,C2 = coordinates_RM2(self,e,i)
        f1,h1 = C1
        f2,h2 = C2
        #Arrow a1 corresponding to e, a2 corresponds to -e
        #Position will be updated from the update_arrow method
        a1 = TorusGaussArrow()
        a1.set_number(m+1)
        a1.set_braidelement(e)
        a1.set_writhe(s)
        a1.set_homotopy(h)
        a1.set_coordinates(C1)
        a2 = TorusGaussArrow()
        a2.set_number(m+2)
        a2.set_braidelement(-e)
        a2.set_writhe(-s)
        a2.set_homotopy(h)
        a2.set_coordinates(C2)
        #Updating the coordinates for the other arrows
        for a in self.get_TorusGaussCode().get_Arrows():
            C = a.get_coordinates()
            if C[0] >= min(f1,h1,f2,h2):
                C[0] = C[0] + 2             
            if C[1] >= min(f1,h1,f2,h2):
                C[1] = C[1] + 2
            if C[0] >= min(max(f1,h1),max(f2,h2)):
                C[0] = C[0] + 2   
            if C[1] >= min(max(f1,h1),max(f2,h2)):
                C[1] = C[1] + 2  
            a.set_coordinates(C)
        self.get_TorusGaussCode().get_Arrows().insert(i,a2)
        self.get_TorusGaussCode().get_Arrows().insert(i,a1)
        self.update_arrows()
        #Updating the Gauss diagram
        self.update_diagram()
        return deepcopy(self)
    def ReidemeisterMoveIII(self,i,j,k):
        '''A method that performs a RM3 for arrows in position i,j and k (Reidemeister Move III)'''
        D1 = deepcopy(self)
        #Getting the 3 arrows to switch
        A = self.get_TorusGaussCode().get_Arrows()
        ai = find_arrow_position(A,i)
        aj = find_arrow_position(A,j)
        ak = find_arrow_position(A,k)
        #Getting their coordinates
        fi,hi = ai.get_coordinates()
        fj,hj = aj.get_coordinates()
        fk,hk = ak.get_coordinates()
        #L is the list of coordinates to be switched
        L = [fi,hi,fj,hj,fk,hk]
        #Operating the switch on the braid
        numb = len(self.get_TorusGaussCode().get_GaussDiagram())
        while len(L) > 0 :
            #For each coordinates, 
            for coord in L:
                a = find_arrow_coord(coord,A)
                for c in a.get_coordinates():
                    if c != coord:
                        coord2 = c
                #We check if it is switchable with only one other coordinate
                if is_switchable_once(coord,coord2,L,numb):
                    Switchables = find_switchables(coord,L,numb)
                    for coord1 in Switchables:
                        if coord1 not in L:
                            Switchables.remove(coord1)
                    #Once we've switch them, we remove them from the list L
                    switch_coordinates(coord,Switchables,A)
                    L.remove(coord)
                    coord3 = Switchables[0]
                    L.remove(coord3)
        #Updating the braid elements in the arrows:
        ei = ai.get_braidelement()
        ej = aj.get_braidelement()
        ek = ak.get_braidelement()
        ei,ej,ek = sign(ei)*abs(ej),sign(ej)*abs(ei),sign(ek)*abs(ej)
        ai.set_braidelement(ei)
        aj.set_braidelement(ej)
        ak.set_braidelement(ek)
        #Updating the position in the arrows
        pi = ai.get_position()
        pk = ak.get_position()
        ai.set_position(k)
        ak.set_position(i)
        A[pi],A[pk] = A[pk],A[pi]
        self.get_TorusGaussCode().set_Arrows(A)
        #Updating the Gauss diagram
        self.update_diagram()
        #Updating the ArrowsNumber 
        Numlist = self.get_ArrowsNumber()
        Numlist[i],Numlist[k] = Numlist[k],Numlist[i]
        self.set_ArrowsNumber(Numlist)
        #Updating the BraidClosure
        BC = self.get_BraidClosure()
        BC[i],BC[j],BC[k] = sign(BC[k])*abs(BC[j]),sign(BC[j])*abs(BC[i]),sign(BC[i])*abs(BC[j])
        self.set_BraidClosure(BC)
        self.update_arrows()
        
        #Generating the singular diagram
        L = [fi,hi,fj,hj,fk,hk]
        A = D1.get_TorusGaussCode().get_Arrows()
        ai = find_arrow_position(A,i)
        ai.set_class('singular')
        aj = find_arrow_position(A,j)
        aj.set_class('singular')
        ak = find_arrow_position(A,k)
        ak.set_class('singular')
        ni = ai.get_number()
        nj = aj.get_number()
        nk = ak.get_number()
        clas = 'RM3 : %d %d %d'%(ni,nj,nk)
        D1.set_Class(clas)
        D1.set_RM3_a(ai)
        D1.set_RM3_b(aj)
        D1.set_RM3_c(ak)
        while len(L) > 0 :
            #For each coordinates, 
            for coord in L:
                a = find_arrow_coord(coord,A)
                for c in a.get_coordinates():
                    if c != coord:
                        coord2 = c
                #We check if it is switchable with only one other coordinate
                if is_switchable_once(coord,coord2,L,numb):
                    Switchables = find_switchables(coord,L,numb)
                    for coord1 in Switchables:
                        if coord1 not in L:
                            Switchables.remove(coord1)
                    #Once we've switch them, we remove them from the list L
                    compute_coordinates(coord,Switchables,A)
                    L.remove(coord)
                    coord3 = Switchables[0]
                    L.remove(coord3)
        #Updating the coordinates for the other arrows
        L = [fi,hi,fj,hj,fk,hk]
        L.sort()
        for a in D1.get_TorusGaussCode().get_Arrows():
            C = a.get_coordinates()
            if C[0] > L[1] and C[0] <= L[2] :
                C[0] = C[0] - 1       
            if C[0] >= L[3] and C[0] <= L[4]:
                C[0] = C[0] - 2
            if C[0] >= L[5]:
                C[0] = C[0] - 3
            if C[1] > L[1] and C[1] <= L[2] :
                C[1] = C[1] - 1       
            if C[1] >= L[3] and C[1] <= L[4]:
                C[1] = C[1] - 2
            if C[1] >= L[5]:
                C[1] = C[1] - 3
            a.set_coordinates(C)
        #Computing the RM3 sign
        lt = local_type(D1)
        gt = global_type(D1)
        D1.set_sign(lt*gt)
        return D1
    def plot_rot(self,GD,figuresize=(6.4, 4.8),figuredpi=100):
        D1 = deepcopy(self)
        roti,rot_sing,Inv = rot(D1,GD)
        for i in range(len(roti)):
            fig = plt.figure(i,figsize=figuresize,dpi=figuredpi,frameon=True)
            roti[i].plot_diagram()
        print('Invariant : {} '.format(Inv))
    def plot_singular_rot(self,GD,figuresize=(6.4, 4.8),figuredpi=100):
        D1 = deepcopy(self)
        roti,rot_sing,Inv = rot(D1,GD)
        for i in range(len(rot_sing)):
            fig = plt.figure(i,figsize=figuresize,dpi=figuredpi,frameon=True)
            rot_sing[i].plot_diagram()
        print('Invariant : {} '.format(Inv))
    def plot_braid_diagram(self,size=0.25,res=200):
        S = self.get_BraidClosure()
        plot_braid(S,size=size,res=res)

### I.b. Generating all the 1-cocycles

In [ ]:
def compute_quarter_direction(ai,trigausscode):
    '''A function that compute the quarter for the Gauss Arrow ai in trigausscode'''
    fa,ha = ai.get_coordinates()
    #Searching for the foot
    for j in range(1,len(trigausscode)):
        if trigausscode[fa-j] == 'A':
            Q1 = 0
            break
        elif trigausscode[fa-j] == 'B':
            Q1 = 1
            break
        elif trigausscode[fa-j] == 'C':
            Q1 = 2
            break
    #Searching for the head
    for j in range(1,len(trigausscode)):
        if trigausscode[ha-j] == 'A':
            Q2 = 0
            break
        elif trigausscode[ha-j] == 'B':
            Q2 = 1
            break
        elif trigausscode[ha-j] == 'C':
            Q2 = 2
            break
    direction = '+'
    if Q1 == Q2 and fa > ha :
            direction = '-'
    return [Q1,Q2],direction

def tri_compute_arrows(trigausscode,homotopy):
    '''A function that computes the set of Gauss Arrows from a TriGaussCode and a Homotopy vector'''
    arrows = []
    if trigausscode != []:
        for i in range(1,(len(trigausscode)-3)/2 + 1):
            ai = TriGaussArrow()
            for j in range(len(trigausscode)):
                if trigausscode[j] == i:
                    foot_ai = j
                elif trigausscode[j] == -i:
                    head_ai = j
            ai.set_coordinates([foot_ai,head_ai])
            ai.set_homotopy(homotopy[i-1])
            ai.set_name(i)
            ai.set_foot(i)
            ai.set_head(-i)
            quarter,direction = compute_quarter_direction(ai,trigausscode)
            ai.set_quarter(quarter)
            ai.set_direction(direction)
            arrows.append(ai)
    return arrows

def circle(center, radius,color):
    ax = plt.gca()
    theta = linspace(0, 2*pi, 100)
    x1 = center[0]+radius*cos(theta)
    x2 = center[1]+radius*sin(theta)
    plt.plot(x1,x2,color)
    plt.axis('scaled')
    return 0

def gauss_diagram(dtype,center,printerr = False):
    #A function that plot a triple point of dtype , dtype must be of format ((int,int),'str') 
    
    #Example of dtype : dtype = ((1,2),'-')
    n = 4
    x = center[0]
    y = center[1]
    circle((x,y),1,'black')
    #Extracting the data of the type
    sign = dtype[2]
    
    #Checking for errors
    err = 1
    if sign != '-' and sign != '+' :
        if printerr == True:
            print('The sign must be - or +')
        err = 0
        return 'Error'
    if err != 0 :
        
        #Computing the homotopy of d
        ml = dtype[0]
        hm = dtype[1]
        if sign == '-' :
            d = ml + hm
        else :
            d = ml + hm - n
            
        #Computing color for ml
        colorml = 'black'
        if ml == 1 :
            colorml = 'red'
        elif ml == 2 :
            colorml = 'blue'
        elif ml == 3 : 
            colorml = 'green'
        else :
            if printerr == True:
                print('Error : ml doesn\'t have a correct homotopy')
        
        #Computing color for hm
        colorhm = 'black'
        if hm == 1 :
            colorhm = 'red'
        elif hm == 2 :
            colorhm = 'blue'
        elif hm == 3 : 
            colorhm = 'green'
        else :
            if printerr == True:
                print('Error : hm doesn\'t have a correct homotopy')
            
        #Computing color for d
        colord = 'black'
        if d == 1 :
            colord = 'red'
        elif d == 2 :
            colord = 'blue'
        elif d == 3 : 
            colord = 'green'
        else :
            if printerr == True:
                print('Error : Such a type doesn\'t exist')
        
        #Computing colors for the arrows
        down = colorml
        if sign =='-' :
            left = colorhm
            right = colord
        else :
            left = colord
            right = colorhm
        
        #Plotting the arrows
        #arrow left
        plt.annotate('', xy = (x,y+1) , xytext = (x+cos(pi*7/6), y+sin(pi*7/6)),
                    arrowprops = {'color': left,'arrowstyle' : '->'})
        #arrow right
        plt.annotate('', xy = (x,y+1), xytext = (x+cos(pi*11/6), y+sin(pi*11/6)),
                    arrowprops = {'color': right,'arrowstyle' : '->'})
        if sign == '-' :
            #arrow down
            plt.annotate('', xy = (x+cos(pi*7/6), y+sin(pi*7/6)), xytext = (x+cos(pi*11/6), y+sin(pi*11/6)),
                        arrowprops = {'color': down,'arrowstyle' : '->'})   
        else :
            #arrow down
            plt.annotate('', xy = (x+cos(pi*11/6), y+sin(pi*11/6)), xytext  = (x+cos(pi*7/6), y+sin(pi*7/6)),
                        arrowprops = {'color': down,'arrowstyle' : '->'})
    return 0

def gauss_diagram_sign(dtype,sign,position,printerr=False) :
    #A function that plots the gauss diagram of a triple point of type dtype and an arrow of homotopy h
    #Q1 and Q2 are the concerned quadrant, direct gives the direction of the arrow (default is +)
    #position is where the sign is plot
    #Example : dtype = ((1,1),'-') , Q1 = 1, Q2 = 1, direct = '-', h = 2
    
    #Plotting the sign
    x = position[0] + 1 - 0.2
    y = position[1] - 0.1
    plt.text(x,y,sign,color='black',size='xx-large')
    
    center = (x+2+0.2 ,y+0.1)
    gauss_diagram(dtype,center,printerr)
    plt.axis('off')
    plt.axis('scaled')
    return 0

def farrow(hp,fp,h,center,printerr = False):
    #plot in an already plot circle of given center, an arrow of homotopy h in position hp,fp
    
    #hp = head position, fp = foot position, h = homotopy
    #Exemple : 1,2,1,(0,1)
    
    #Testing for errors in positions
    err = 1
    if hp == fp :
        if printerr == True:
            print("Head and foot cannot be in the same point")
        err = 0
        return 'Error'
    if hp == 11 or hp == 3 or hp == 7 or fp == 11 or fp == 3 or fp == 7 :
        if printerr == True:
            print("Arrow cannot be added on the singular points")
        err = 0
        return 'Error'
    if err != 0 :
        x = center[0]
        y = center[1]
        #Computing the head and foot position (ha = head angle, fa = foot angle)
        head = (x+cos(hp),y+sin(hp))
        foot = (x+cos(fp),y+sin(fp))
        #Determining the color
        color = 'black'
        if h == 1 :
            color = 'red'
        if h == 2 :
            color = 'blue'
        if h == 3 :
            color = 'green'
        
        #Plotting the arrow
        plt.annotate('', xy = head, xytext  = foot,
                        arrowprops = {'color': color,'arrowstyle' : '->'})
    return 0

def compute_ends_for_quarters(trigausscode):
    '''A function that computes for each quarter of a Gauss Diagram the number of ends'''
    #Quarter 0
    nb0 = 0
    for i in range(1,len(trigausscode)):
        if trigausscode[i] == 'B':
            break
        nb0 = nb0 + 1
    #Quarter 1
    nb1 = 0 
    for i in range(nb0+2,len(trigausscode)):
        if trigausscode[i] == 'C':
            break
        nb1 = nb1 + 1
    #Quarter 2
    nb2 = len(trigausscode) - 3 - nb0 - nb1
    return nb0,nb1,nb2

def compute_position(arrow,nb0,nb1,nb2):
    '''A function that computes the position of Gauss Arrow arrow for plotting'''
    #nb0, nb1 and nb2 are the number of ends in the quarters
    #Getting the coordinates
    fc,hc = arrow.get_coordinates()
    Q1,Q2 = arrow.get_quarter()
    #if foot is in Q0
    if Q1 == 0:
        n0 = nb0 + 1
        fp = 11*pi/6 + fc*2*pi/(3*n0)
    #if foot is in Q1
    elif Q1 == 1:
        n1 = nb1 + 1 
        fc = fc - nb0 - 1
        fp = pi/2 + fc*2*pi/(3*n1)
    #if foot is in Q2
    elif Q1 == 2:
        n2 = nb2 + 1
        fc = fc - nb0 - 1 - nb1 - 1 
        fp = 7*pi/6 + fc*2*pi/(3*n2)
    #if head is in Q0
    if Q2 == 0:
        n0 = nb0 + 1
        hp = 11*pi/6 + hc*2*pi/(3*n0)
    #if head is in Q1
    elif Q2 == 1:
        n1 = nb1 + 1 
        hc = hc - nb0 - 1
        hp = pi/2 + hc*2*pi/(3*n1)
    #if head is in Q2
    elif Q2 == 2:
        n2 = nb2 + 1
        hc = hc - nb0 - 1 - nb1 - 1 
        hp = 7*pi/6 + hc*2*pi/(3*n2)
    return fp,hp

def compute_coordinates_sing(dtype,trigausscode):
    '''A function that computes the coordinates for the singular arrows in a TriGaussDiagram'''
    sign = dtype[2]
    #Getting the coordinates for B and C
    A = 0
    for i in range(len(trigausscode)):
        if trigausscode[i] == 'B':
            B = i
        elif trigausscode[i] == 'C':
            C = i
    #Computing the coordinates for the singular arrows
    fd = C
    hd = B
    fhm = A
    hhm = B
    fml = C
    hml = A
    if sign == '-':
        fml,hml = hml,fml
        fhm,fd = fd,fhm
    return [fd,hd],[fhm,hhm],[fml,hml]

def tri_compute_trigausscode(arrows):
    '''A function that computes a trigausscode from a set of arrows'''
    trigausscode =  l = [0] * (2*len(arrows) + 3)
    for a in arrows:
        i = a.get_name()
        fa,ha = a.get_coordinates()
        trigausscode[fa] = i
        trigausscode[ha] = -i
    trigausscode[0] = 'A'
    for i in range(len(trigausscode)):
        if trigausscode[i] == 0:
            trigausscode[i] = 'B'
            break
    for i in range(len(trigausscode)):
        if trigausscode[i] == 0:
            trigausscode[i] = 'C'
            break 
    return trigausscode

def tri_compute_trigausscode_braid(arrows):
    '''A function that computes a trigausscode from a set of arrows'''
    trigausscode =  l = [0] * (2*len(arrows) + 3)
    for a in arrows:
        i = a.get_number()
        fa,ha = a.get_coordinates()
        trigausscode[fa] = i
        trigausscode[ha] = -i
    trigausscode[0] = 'A'
    for i in range(len(trigausscode)):
        if trigausscode[i] == 0:
            trigausscode[i] = 'B'
            break
    for i in range(len(trigausscode)):
        if trigausscode[i] == 0:
            trigausscode[i] = 'C'
            break 
    return trigausscode

In [ ]:
class TriGaussArrow:
    "An arrow in a Gauss diagram of a triple point"
    def __init__(self,coordinates=('fc','hc'),quarter=('INIT','INIT'),direction='INIT',homotopy=0,name='default',foot='INIT',head='INIT',nomenclature='INIT'):
        self.__coordinates = coordinates
        self.__quarter = quarter
        self.__direction = direction
        self.__homotopy = homotopy
        self.__name = name
        self.__foot = foot
        self.__head = head
        self.__nomenclature = nomenclature
        self.update()
    def update(self):
        self.__nomenclature = [self.get_name(),self.get_coordinates(),self.get_homotopy()]
        if type(self.get_name()) == int:
            self.__nomenclature.append(self.get_quarter())
        if self.get_quarter()[0] == self.get_quarter()[1]:
            self.__nomenclature.append(self.get_direction())
    def set_name(self,name):
        self.__name = name
        self.update()
    def set_foot(self,foot):
        self.__foot = foot
    def set_head(self,head):
        self.__head = head
    def set_coordinates(self,coordinates):
        self.__coordinates = coordinates
        self.update()
    def set_quarter(self,quarter):
        self.__quarter = quarter
        self.update()
    def set_direction(self,direction):
        self.__direction = direction
        self.update()
    def set_homotopy(self,h):
        self.__homotopy = h
        self.update()
    def get_coordinates(self):
        return self.__coordinates
    def get_quarter(self):
        return self.__quarter
    def get_direction(self):
        return self.__direction
    def get_homotopy(self):
        return self.__homotopy
    def get_name(self):
        return self.__name
    def get_foot(self):
        return self.__foot
    def get_head(self):
        return self.__head
    def get_nomenclature(self):
        return self.__nomenclature
    def get_infos(self):
        print('Nomenclature :',self.get_nomenclature())
        print('Name :',self.get_name())
        print('Coordinates :',self.get_coordinates())
        print('Foot : {}, Head : {}'.format(self.get_foot(),self.get_head()))
        print('Homotopy class :',self.get_homotopy())
        print('Quarters :',self.get_quarter())
        if self.get_quarter()[0] == self.get_quarter()[1]:
                print('Direction :',self.get_direction)
    def __eq__(self, A):
        if A.__class__.__name__ != "TriGaussArrow" :
            print('Error : Could not test the operand ==. Unsupported type for the second argument')
            return False
        else :
            if self.get_coordinates() == A.get_coordinates() and self.get_name() == A.get_name() and self.get_homotopy() == A.get_homotopy():
                return True
            else :
                return False
    def __str__(self):
        return str(self.get_nomenclature())
    def __repr__(self):
        return str(self)
    
class CoeffGaussDiagramTri:
    "A signed Gauss Diagram of a triple point"
    def __init__(self,dtype=[0,0,'INIT'],trigausscode=[],homotopy=[],coeff = 0,d = [1,1,'+','d'],hm = [1,1,'+','hm'],ml = [1,1,'+','ml'],printerrors = False):
        #Checking for errors
        if len(trigausscode) == 2*len(homotopy)+3 or (trigausscode == [] and homotopy == []):
            self.__type = dtype
            self.__arrows = tri_compute_arrows(trigausscode,homotopy)
            self.__coefficient = coeff
            self.__printerrors = printerrors
            self.__d = d
            self.__hm = hm
            self.__ml = ml
            self.__trigausscode = trigausscode
            self.__homotopy = homotopy
            if (trigausscode != [] or homotopy != []):
                self.update_type()
                self.compute_name()
            else:
                self.__name = 'INIT'
        else:
            print('Error : Gauss Code and Homotopy vector don\'t have same length')
    def compute_name(self):
        if self.get_coefficient() == -1:
            sgn = '-'
        elif self.get_coefficient() == 1:
            sgn = '+'
        else:
            sgn = self.get_coefficient()
        self.__name = '%s (%s,%s,%s)'%(sgn,self.get_type(),self.get_trigausscode(),self.get_homotopy())
    def update_type(self):
        self.__d = TriGaussArrow()
        self.__hm = TriGaussArrow()
        self.__ml = TriGaussArrow()
        ml,hm,sign = self.get_type()
        if sign == '-' :
            self.__d.set_homotopy(ml + hm)
            self.__d.set_foot('A')
            self.__d.set_head('B')
            self.__hm.set_foot('C')
            self.__hm.set_head('B')
            self.__ml.set_foot('A')
            self.__ml.set_head('C')
        else :
            self.__d.set_homotopy(ml + hm - Nbraid)
            self.__d.set_foot('C')
            self.__d.set_head('B')
            self.__hm.set_foot('A')
            self.__hm.set_head('B')
            self.__ml.set_foot('C')
            self.__ml.set_head('A')
        coordd,coordhm,coordml = compute_coordinates_sing(self.get_type(),self.get_trigausscode())
        self.__d.set_name('d')
        self.__d.set_direction(sign)
        self.__d.set_coordinates(coordd)
        self.__d.set_quarter(['d','d'])
        self.__hm.set_homotopy(hm)
        self.__hm.set_name('hm')
        self.__hm.set_direction(sign)
        self.__hm.set_coordinates(coordhm)
        self.__hm.set_quarter(['hm','hm'])
        self.__ml.set_homotopy(ml)
        self.__ml.set_name('ml')
        self.__ml.set_direction(sign)
        self.__ml.set_coordinates(coordml)
        self.__ml.set_quarter(['ml','ml']) 
    def set_type(self,ml,hm,sign):
        self.__type = [ml,hm,sign]
        self.update_type()
        self.compute_name()   
    def set_arrows(self,arrows):
        self.__arrows = arrows
        self.compute_name()
    def set_coefficient(self,coeff):
        self.__coefficient = coeff
        self.compute_name()
    def set_printerrors(self,printerr):
        self.__printerrors = printerr
    def set_trigausscode(self,trigausscode):
        self.__trigausscode = trigausscode
    def set_homotopy(self,homotopy):
        self.__homotopy = homotopy
    def get_type(self):
        return self.__type
    def get_arrows(self):
        return self.__arrows
    def get_coefficient(self):
        return self.__coefficient
    def get_d(self):
        return self.__d
    def get_hm(self):
        return self.__hm
    def get_ml(self):
        return self.__ml
    def get_name(self):
        return self.__name
    def get_printerrors(self):
        return self.__printerrors
    def get_trigausscode(self):
        return self.__trigausscode
    def get_homotopy(self):
        return self.__homotopy
    def get_arrowsinfos(self):
        for a in self.get_arrows():
            a.get_infos()
    def get_infos(self):
        print('Type : ',self.get_type())
        print('Coefficient : ',self.get_coefficient())
        print('d : ',self.get_d())
        print('hm : ',self.get_hm())
        print('ml : ',self.get_ml())
        print('Name : ',self.get_name())
        print('Printerrors : ',self.get_printerrors())
        self.get_arrowsinfos()
    def plot(self,position=(0,0),solo=True,nbfigure='INIT'):
        if self.get_coefficient() != 0:
            if nbfigure != 'INIT':
                fig = plt.figure(nbfigure)
            #Updating the sign to plot
            if self.get_coefficient() == -1 :
                sign = '-'
            elif self.get_coefficient() == +1:
                sign = '+'
            elif self.get_coefficient() > 0 :
                sign = '+ {}'.format(self.get_coefficient())
            else:
                sign = self.get_coefficient()
            #Plotting the triple point
            dtype = self.get_type()
            printerr = self.get_printerrors()
            gauss_diagram_sign(dtype,sign,position,printerr)
            #Computing number of ends for each quarter
            nb0,nb1,nb2 = compute_ends_for_quarters(self.get_trigausscode())
            #Plotting the arrows
            x = position[0] + 3
            y = position[1] 
            for arrow in self.get_arrows():
                #Computing the positions
                fp,hp = compute_position(arrow,nb0,nb1,nb2)
                h = arrow.get_homotopy()
                farrow(hp,fp,h,[x,y],printerr)
            if solo == True:
                ax = plt.gca()
                xlim = plt.xlim(position[0],position[0]+4.1)
                ylim = plt.ylim(position[1]-1.2,position[1]+1.2)
        elif self.get_printerrors():
            print('{} is empty'.format(self))
    def __eq__(self, D):
        if D == 0:
            return self.get_coefficient() == 0
        elif D.__class__.__name__ != "CoeffGaussDiagramTri" :
            print('Error : Could not test the operand ==. Unsupported type for the second argument')
            return False
        else:
            if self.get_type() == D.get_type() and self.get_arrows() == D.get_arrows() and self.get_coefficient() == D.get_coefficient():
                return True
            else :
                return False
    def __add__(self,GD):
        LE = LinearCombinationOfTriGaussDiagrams(self)
        if GD.__class__.__name__ == "CoeffGaussDiagramTri" or GD.__class__.__name__ == "LinearCombinationOfTriGaussDiagrams":
            LE = LE + GD
            return LE
        elif GD == 0:
            return LE
        else:
            return 'ERROR not the right type for an addition'
    def __neg__(self):
        GD = deepcopy(self)
        GD.set_coefficient(-1*self.get_coefficient())
        return GD
    def __sub__(self,GD):
        return self + (-GD)
    def __str__(self):
        return str(self.get_name())
    def __repr__(self):
        return str(self)  
    def __len__(self):
        return len(self.get_arrows())

    
class LinearCombinationOfTriGaussDiagrams:
    "A linear combination of Gauss Diagrams of a triple point containing one arrow"
    def __init__(self,GD=0,printerrors = False,name = 'INIT',printname = False):
        if GD == 0:
            self.__expression = []
        elif GD.__class__.__name__ == "CoeffGaussDiagramTri":
            self.__expression = [GD]
        else:
            self.__expression = GD
        self.__correction = []
        self.__printerrors = printerrors
        self.__name = name
        self.__printname = printname
    def set_expression(self,expression):
        self.__expression = expression
    def set_correction(self,expression):
        self.__correction = expression
    def set_printerrors(self,printerr):
        self.__printerrors = printerr
    def set_name(self,name):
        self.__name = name
    def set_printname(self,printname):
        self.__printname = printname
    def get_expression(self):
        return self.__expression
    def get_correction(self):
        return self.__correction
    def get_printerrors(self):
        return self.__printerrors
    def get_name(self):
        return self.__name
    def get_printname(self):
        return self.__printname
    def get_infos(self,i=1):
        if self.get_printname() :
            print(self.get_name())
        #Get the infos of the i-th Gauss Diagram (i starting at 1)
        n = len(self)
        i = i - 1
        if i>= n :
            print('Error : there isn\'t a {}-th Gauss diagram'.format(i+1))
        else :
            self[i].get_infos()
        print('Correction',self.get_correction())
    def clean_diagrams(self):
        for GD in self.get_expression():
            if GD.get_coefficient() == 0:
                self.__expression.remove(GD)
    def plot(self,figurenum=0,expression_length=5):
        n = len(self)
        if int(len(self)/expression_length) == 0:
            heigh = 1
        else:
            heigh = int(len(self)/expression_length)
        if n > 0 :
            if not self.get_printname():
                fig = plt.figure(figurenum,figsize=(expression_length*6, 5*heigh))
                i = 0
                j = 0
                for i in range(n) :
                    iline = int(i/expression_length)*(-2.5)
                    self[i].plot((j,iline),False)
                    j = (j+4)%(expression_length*4)
                if self.get_correction() != []:
                    j = 1
                    for i in range(len(self.get_correction())):
                        cor = self.get_correction()[i]
                        plt.text(4*n+j,-0.1,r'$+$',color='black',size='xx-large')
                        plotcor = r'$ \frac{1}{2}(w(%s)-1)$'%cor
                        plt.text(4*n+j+2,0-0.1,plotcor,color='black',size='xx-large',horizontalalignment='center')
                        j = j + 4
                plt.xlim(0,0.1+4*expression_length)
                plt.ylim(int(len(self)/expression_length)*(-2.5)-1.2,1.2)
                ax = plt.axis('off')
            else:
                fig = plt.figure(figurenum,figsize=(expression_length*6, 5*heigh))
                j = 0
                x = - 0.2
                y = - 0.1
                plt.text(x,y,self.get_name(),color='black',size='xx-large')
                plt.text(x+1,y,'=',color='black',size='xx-large')
                for i in range(n) :
                    iline = int(i/expression_length)*(-2.5)
                    self[i].plot((j+1,iline),False)
                    j = (j+4)%(expression_length*4)
                if self.get_correction() != []:
                    j = 2
                    for i in range(len(self.get_correction())):
                        cor = self.get_correction()[i]
                        plt.text(4*n+j,-0.1,r'$+$',color='black',size='xx-large')
                        plotcor = r'$ \frac{1}{2}(w(%s)-1)$'%cor
                        plt.text(4*n+j+2,0-0.1,plotcor,color='black',size='xx-large',horizontalalignment='center')
                        j = j + 4
                plt.xlim(0,1.1+4*expression_length)
                plt.ylim(int(len(self)/expression_length)*(-2.5)-1.2,int(len(self)/expression_length)+1.2)
                ax = plt.axis('off')
        else :
            if self.get_printerrors() :
                print('Combination is empty' )                      
    def append(self,GD):
        self.__expression.append(GD)
    def __len__(self):
        return len(self.get_expression())
    def __getitem__(self, key):
        return self.get_expression()[key]
    def __setitem__(self, key, value):
        self.__expression[key] = value
    def __delitem__(self, key):
        self.__expression.pop(key)
    def __missing__(self, key):
        print('%i is not a valid index'% key)
    def __iter__(self):
        return iter(self.get_expression())
    def __reversed__(self):
        return reversed(self.get_expression())
    def __contains__(self, item):
        ans = False
        for GD in self:
            if GD == item:
                ans = True
        return ans
    def __add__(self,GD):
        LE = deepcopy(self)
        if GD.__class__.__name__ == "LinearCombinationOfTriGaussDiagrams":
            for g in GD:
                LE = LE + g
            return LE
            
        elif GD.__class__.__name__ == "CoeffGaussDiagramTri" and GD.get_coefficient() != 0:
            isalready_in = False
            for i in range(len(self.get_expression())):
                g = self.get_expression()[i]
                if g.get_type() == GD.get_type() and g.get_arrows() == GD.get_arrows() and g.get_trigausscode() == GD.get_trigausscode() and g.get_homotopy() == GD.get_homotopy():
                    isalready_in = True
                    k = i
                    break
            if isalready_in :
                coeff1 = self.__expression[k].get_coefficient()
                coeff2 = GD.get_coefficient()
                expression = deepcopy(self.get_expression())
                expression[k].set_coefficient(coeff1+coeff2)
                LE = LinearCombinationOfTriGaussDiagrams(expression)
            else:
                LE.append(GD)
                LE.set_name('%s + %s'%(LE,GD))
            return LE
        elif GD == 0:
            return LE
        elif GD == LinearCombinationOfTriGaussDiagrams():
            return LE
        elif type(GD) == str:
            LE.__correction.append(GD)
            return LE
        else :
            print('Error : wrong format for',GD)
            return 'ERROR'
    def __neg__(self):
        LE = deepcopy(self)
        for GD in LE:
            GD = -GD
        return LE
    def __sub__(self,GD):
        return self + (-GD)
    def __str__(self):
        pr = ''
        for GD in self:
            pr = pr+GD.get_name()
        return pr
    def __repr__(self):
        return str(self) 

In [ ]:
def generate_reduc_gaussdiagram(arrows,trigausscode):
    '''A function that generates a reducted Gauss Code that contains only listed arrows'''
    #Generating a reduct GaussDiagram
    copytrigausscode = deepcopy(trigausscode)
    allowedset = []
    for a in arrows:
        namea = a.get_name()
        if type(namea) == int:
            allowedset.append(namea)
            allowedset.append(-namea)
        else:
            fs,hs = a.get_coordinates()
            allowedset.append(copytrigausscode[fs])
            allowedset.append(copytrigausscode[hs])
    reductrigausscode = []
    for c in copytrigausscode:
        if c in allowedset:
            reductrigausscode.append(c)   
    return reductrigausscode

def arrows_in_circle(arrows,trigausscode):
    '''A function that detects if a set of TriGaussArrow are in a circle configuration'''
    ans = True
    #Generating a reduct GaussDiagram
    reductrigausscode = generate_reduc_gaussdiagram(arrows,trigausscode)
    #For each arrow, if we start from i or -i, the next one should be -i or i except for singular arrows   
    for a in arrows:
        namea = a.get_name()
        fc,hc = a.get_coordinates()
        if type(namea) == int:
            if fc < hc:
                firstend = namea
            else:
                firstend = -namea
            for i in range(len(reductrigausscode)):
                if reductrigausscode[i] == firstend:
                    break
            if i+1 == len(reductrigausscode):
                endafteri = reductrigausscode[0]
            else:
                endafteri = reductrigausscode[i+1]
            endbeforei = reductrigausscode[i-1]
            if endafteri != -firstend and endbeforei != -firstend:
                ans = False
        else:
            for i in range(len(reductrigausscode)):
                if type(reductrigausscode[i]) == str:
                    break
            if i+1 == len(reductrigausscode):
                endafteri = reductrigausscode[0]
            else:
                endafteri = reductrigausscode[i+1]
            endbeforei = reductrigausscode[i-1]
            if type(endafteri) != str and type(endbeforei) != str :
                ans = False
    return ans

def dist(a,b,total=12):
    distance = min(abs((b-a)%total),abs((a-b)%total))
    return distance

def shortest_distance(a,b,trigausscode):
    '''A function that returns the shortest way from a to b (or from b to a) in a TriGaussCode'''
    #Length from a to b
    for i in range(len(trigausscode)):
        if trigausscode[i] == a:
            break
    lenab = 0
    while trigausscode[i] != b:
        i = i+1 
        if i == len(trigausscode):
            i = 0
        lenab = lenab + 1
    #Length from b to a
    for i in range(len(trigausscode)):
        if trigausscode[i] == b:
            break
    lenba = 0
    while trigausscode[i] != a:
        i = i+1 
        if i == len(trigausscode):
            i = 0
        lenba = lenba + 1
    #Computing shortest way
    if lenab < lenba :
        return a,b
    else:
        return b,a

def border_homotopy_arrow(arrow,reductrigausscode):
    '''A function that returns the border homotopy class of an arrow in a circle configuration'''
    h = arrow.get_homotopy()
    #Looking for the shortest way:
    foot = arrow.get_foot()
    head = arrow.get_head()
    if foot == 'INIT' or head == 'INIT':
        print('ERROR : foot or head is not defined for the arrow {}'.format(arrow.get_name()))
        return 'Error'
    else:
        startway,endway = shortest_distance(foot,head,reductrigausscode)
        if startway == foot:
            return Nbraid - h
        else:
            return h
        
def arrows_in_circle(arrows,trigausscode,printerr=False):
    '''A function that detects if a set of TriGaussArrow are in a circle configuration'''
    ans = True
    #Generating a reduct GaussDiagram
    reductrigausscode = generate_reduc_gaussdiagram(arrows,trigausscode)
    #For each arrow, if we start from i or -i, the next one should be -i or i except for singular arrows   
    for a in arrows:
        namea = a.get_name()
        fc,hc = a.get_coordinates()
        #Regular arrows
        if type(namea) == int:
            if fc < hc:
                firstend = namea
            else:
                firstend = -namea
            for i in range(len(reductrigausscode)):
                if reductrigausscode[i] == firstend:
                    break
            if i+1 == len(reductrigausscode):
                endafteri = reductrigausscode[0]
            else:
                endafteri = reductrigausscode[i+1]
            endbeforei = reductrigausscode[i-1]
            if endafteri != -firstend and endbeforei != -firstend:
                ans = False
        #Singular arrows
        else:
            for i in range(len(reductrigausscode)):
                if type(reductrigausscode[i]) == str:
                    break
            if i+1 == len(reductrigausscode):
                endafteri = reductrigausscode[0]
            else:
                endafteri = reductrigausscode[i+1]
            endbeforei = reductrigausscode[i-1]
            if type(endafteri) != str and type(endbeforei) != str :
                ans = False
    return ans

def circle_homotopy(arrows,trigausscode):
    '''A function that returns the homotopy of the inner loop associated to a set of arrows'''
    reductrigausscode = generate_reduc_gaussdiagram(arrows,trigausscode)
    inner_homotopy = Nbraid
    for arrow in arrows:
        inner_homotopy = inner_homotopy - border_homotopy_arrow(arrow,reductrigausscode)
    return inner_homotopy

def test_n_arrows(arrows,trigausscode,printerr=False):
    '''A function that tests if a configuration of arrows is allowed in a trigausscode'''
    ans = True
    #Testing if the n arrows are in circle
    if arrows_in_circle(arrows,trigausscode,printerr) and arrows != []:
        #Testing that the inner loop homotopy
        if circle_homotopy(arrows,trigausscode) not in range(1,Nbraid):
            ans = False
            if printerr:
                print('      The arrows {} are in circle but inner loop homotopy is wrong : {}'.format(arrows,circle_homotopy(arrows,trigausscode)))
    #Testing all n-1 set of arrows
    if printerr:
        print('      Test of subset of n-1 arrows of {}'.format(arrows))
    for a in arrows:
        #Checking a subset without a
        subsetarrows = deepcopy(arrows)
        subsetarrows.remove(a)
        if test_n_arrows(subsetarrows,trigausscode,printerr) == False:
            ans = False
            if printerr:
                print('         The subset of arrows {} is not allowed'.format(subsetarrows))
    return ans

def is_diagram_allowed(D,printerr=False):
    '''A function that computes if a CoeffGaussDiagramTri is allowed'''
    trigausscode = D.get_trigausscode()
    d = D.get_d()
    hm = D.get_hm()
    ml = D.get_ml()
    arrows = D.get_arrows()
    arrowsd = deepcopy(arrows)
    arrowsd.append(d)
    arrowshm = deepcopy(arrows)
    arrowshm.append(hm)
    arrowsml = deepcopy(arrows)
    arrowsml.append(ml)
    if printerr:
        print('Test of diagram',D)
        print('   Test of n arrows with d : ',test_n_arrows(arrowsd,trigausscode))
        boold = test_n_arrows(arrowsd,trigausscode,printerr)
        print('   Test of n arrows with hm : ',test_n_arrows(arrowshm,trigausscode))
        boolhm = test_n_arrows(arrowshm,trigausscode,printerr)
        print('   Test of n arrows with ml : ',test_n_arrows(arrowsml,trigausscode,printerr))
    return (D.get_coefficient() !=0 and test_n_arrows(arrowsd,trigausscode) and test_n_arrows(arrowshm,trigausscode) and test_n_arrows(arrowsml,trigausscode))


def all_insert_possible(e,trigausscode):
    '''A function that lists all the possible insertions of the element e in a trigausscode'''
    ListPoss = []
    for i in range(1,len(trigausscode)+1):
        newtrigausscode = deepcopy(trigausscode)
        newtrigausscode.insert(i,e)
        ListPoss.append(newtrigausscode)
    return ListPoss

def list_all_possible_trigausscode(nbarrows):
    '''A function that returns a list of all possible trigausscodes with nbarrows arrows'''
    InitList = [['A','B','C']]
    for i in range(1,nbarrows+1):
        ListTemp = []
        #Generating all the gausscode for the foot i
        for trigausscode in InitList:
            ListTemp.extend(all_insert_possible(i,trigausscode))
        #Generating all the gausscode for the head -i
        InitList = []
        for trigausscode in ListTemp:
            InitList.extend(all_insert_possible(-i,trigausscode))
    return InitList

def list_all_possible_CoeffGaussDiagramTri(nbarrows,dtypelist=[1,1,'-'],maxi = 50000):
    '''A function that lists all possible CoeffGaussDiagramTri'''
    ListAll = []
    ListAllPossibleTGC = list_all_possible_trigausscode(nbarrows)
    #Generating for each type
    for dtype in ListType:
        if dtype != dtypelist:
            continue
        #Generating the Homotopy vector for each type of arrows
        pchoice = range(1,Nbraid)
        if nbarrows == 1 :
            choice = []
            for c in pchoice:
                choice.append([c])
            ListHom =  choice
        else:
            homotopy_choice = range(1,Nbraid)
            choice = []
            for c in homotopy_choice:
                choice.append([c])
            n = len(choice)
            L_temp = choice
            for i in range(nbarrows-1):
                ListHom = []
                for k in range(len(L_temp)):
                    l_k = L_temp[k]
                    for c in choice:
                        temp = deepcopy(l_k)
                        temp.extend(c)
                        ListHom.append(temp)
                L_temp = deepcopy(ListHom)
        #Removing "same" homotopy vector
        Notallowed = []
        for Hom in ListHom:
            for i in range(len(Hom)-1):
                if Hom[i] > Hom[i+1]:
                    Notallowed.append(Hom)
        for Hom in Notallowed:
            if Hom in ListHom:
                ListHom.remove(Hom)
        #For each homotopy vector, we list all possible trigausscodes
        for Hom in ListHom:
            for TGC in ListAllPossibleTGC:
                D = CoeffGaussDiagramTri(dtype,TGC,Hom,1)
                #Appending only allowed diagrams
                if is_diagram_allowed(D):
                    ListAll.append(D)
                if len(ListAll) >= maxi:
                    break
            if len(ListAll) >= maxi:
                break
    return ListAll

def list_all_possible_homogene_CoeffGaussDiagramTri(nbarrows,dtypelist=[1,1,'-'],maxi = 50000):
    '''A function that lists all possible CoeffGaussDiagramTri'''
    ListAll = []
    ListAllPossibleTGC = list_all_possible_trigausscode(nbarrows)
    #Generating for each type
    for dtype in ListType:
        if dtype != dtypelist:
            continue
        #Generating the Homotopy vector for each type of arrows
        pchoice = range(1,Nbraid)
        if nbarrows == 1 :
            choice = []
            for c in pchoice:
                choice.append([c])
            ListHom =  choice
        else:
            homotopy_choice = range(1,Nbraid)
            choice = []
            for c in homotopy_choice:
                choice.append([c])
            n = len(choice)
            L_temp = choice
            for i in range(nbarrows-1):
                ListHom = []
                for k in range(len(L_temp)):
                    l_k = L_temp[k]
                    for c in choice:
                        temp = deepcopy(l_k)
                        temp.extend(c)
                        ListHom.append(temp)
                L_temp = deepcopy(ListHom)
        #Removing "same" homotopy vector
        Notallowed = []
        for Hom in ListHom:
            for i in range(len(Hom)-1):
                if Hom[i] >= Hom[i+1]:
                    Notallowed.append(Hom)
        for Hom in Notallowed:
            if Hom in ListHom:
                ListHom.remove(Hom)
        #For each homotopy vector, we list all possible trigausscodes
        for Hom in ListHom:
            for TGC in ListAllPossibleTGC:
                D = CoeffGaussDiagramTri(dtype,TGC,Hom,1)
                #Appending only allowed diagrams
                if is_diagram_allowed(D):
                    ListAll.append(D)
                if len(ListAll) >= maxi:
                    break
            if len(ListAll) >= maxi:
                break
    return ListAll

def list_all_allowed_homogene_CoeffGaussDiagramTri(nbarrows,dtype=[1,1,'-'],maxi=50000):
    '''A function that lists all allowed homogene CoeffGaussDiagramTri'''
    ListAll = list_all_possible_CoeffGaussDiagramTri(nbarrows,dtype,maxi)
    #Removing all the non homogene CoeffGaussDiagramTri
    nonAllowedList = []
    for D in ListAll:
        Hom = D.get_homotopy()
        #Checking if two arrows have same homotopy
        deleteHom = False
        for i in range(len(Hom)):
            for j in range(i+1,len(Hom)):
                if Hom[i] == Hom[j]:
                    deleteHom = True
        if deleteHom == True or is_diagram_allowed(D) != True:
            nonAllowedList.append(D)
    #Removing the non allowed SignedGaussDiagramTri
    for D in nonAllowedList:
        ListAll.remove(D)
    return ListAll

def list_all_allowed_homogene_CoeffGaussDiagramTri(nbarrows,dtype=[1,1,'-'],maxi=50000):
    '''A function that lists all allowed homogene CoeffGaussDiagramTri'''
    ListAll = list_all_possible_homogene_CoeffGaussDiagramTri(nbarrows,dtype,maxi)
    #Removing all the non homogene CoeffGaussDiagramTri
    nonAllowedList = []
    for D in ListAll:
        Hom = D.get_homotopy()
        #Checking if two arrows have same homotopy
        deleteHom = False
        for i in range(len(Hom)):
            for j in range(i+1,len(Hom)):
                if Hom[i] == Hom[j]:
                    deleteHom = True
        if deleteHom == True or is_diagram_allowed(D) != True:
            nonAllowedList.append(D)
    #Removing the non allowed SignedGaussDiagramTri
    for D in nonAllowedList:
        ListAll.remove(D)
    return ListAll

In [ ]:
def distance_from_to(a,b,trigausscode):
    '''A function that computes the distance from a to b in a trigausscode'''
    #Looking for a and b
    for i in range(len(trigausscode)):
        if trigausscode[i] == a:
            pa = i
        if trigausscode[i] == b:
            pb = i
    total = len(trigausscode)
    return (pb-pa)%total
    
def switch_ends(a,b,trigausscode):
    '''A function that switches a and b in a trigausscode'''
    #Finding a and b
    for i in range(len(trigausscode)):
        if trigausscode[i] == a:
            pa = i
        if trigausscode[i] == b:
            pb = i
    newtri = deepcopy(trigausscode)
    #Switching regular case
    if pa != len(trigausscode)-1 and pb != len(trigausscode)-1and pa != 0 and pb != 0:
        newtri[pa],newtri[pb] = newtri[pb],newtri[pa]
    #a is at the end
    elif pa == len(trigausscode)-1:
        newtri.remove(a)
        newtri.insert(1,a)
    #b is at the end
    elif pb == len(trigausscode)-1:
        newtri.remove(b)
        newtri.insert(1,b) 
    #a is in first place
    elif pa == 0:
        newtri.remove(b)
        newtri.append(b)
    elif pb == 0:
        newtri.remove(a)
        newtri.append(a)
    return newtri

def positive_slide_move_head_is_possible(D,arrow,printerr=False):
    '''A function that detects if an arrow can perform a positive slide move for its head'''
    trigausscode = D.get_trigausscode()
    namea = arrow.get_name()
    #Checking if -namea is close to 'A', 'B' or 'C'
    if distance_from_to(-namea,'A',trigausscode) == 1 or distance_from_to(-namea,'B',trigausscode) == 1 or distance_from_to(-namea,'C',trigausscode) == 1:
        return True
    else:
        return False

def slide_move_positive_head(D1,arrow,printerr=False):
    '''A function that performs a positive slide move for the head of an arrow'''
    #Input : D1 a SignedGaussDiagramWithOneArrow
    #Output : D2 a SignedGaussDiagramWithOneArrow obtained from performing a slide move with the head of the arrow on D1 or 0 if not possible
    
    #Sliding if possible
    if positive_slide_move_head_is_possible(D1,arrow,printerr):
        trigausscode = deepcopy(D1.get_trigausscode())
        dtype = D1.get_type()
        homotopy = D1.get_homotopy()
        coeff = D1.get_coefficient()
        #Finding the singular point to switch
        fc,hc = arrow.get_coordinates()
        maxi = len(trigausscode)
        singpt = (hc+1)%maxi
        trigausscode[hc],trigausscode[singpt] = trigausscode[singpt],trigausscode[hc]
        while trigausscode[0] != 'A':
            trigausscode = trigausscode[1:] + trigausscode[:1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
    else:
        if printerr:
            print('Error : cannot perform a positive head slide move')
        D2 = CoeffGaussDiagramTri()
        D2.set_coefficient(0)
    return D2

def negative_slide_move_head_is_possible(D,arrow,printerr=False):
    '''A function that detects if an arrow can perform a positive slide move for its head'''
    trigausscode = D.get_trigausscode()
    namea = arrow.get_name()
    #Checking if -namea is close to 'A', 'B' or 'C'
    if distance_from_to('A',-namea,trigausscode) == 1 or distance_from_to('B',-namea,trigausscode) == 1 or distance_from_to('C',-namea,trigausscode) == 1:
        return True
    else:
        return False
    
def slide_move_negative_head(D1,arrow,printerr=False):
    '''A function that performs a positive slide move for the head of an arrow'''
    #Input : D1 a SignedGaussDiagramWithOneArrow
    #Output : D2 a SignedGaussDiagramWithOneArrow obtained from performing a slide move with the head of the arrow on D1 or 0 if not possible

    #Sliding if possible
    if negative_slide_move_head_is_possible(D1,arrow,printerr):
        trigausscode = deepcopy(D1.get_trigausscode())
        dtype = D1.get_type()
        homotopy = D1.get_homotopy()
        coeff = D1.get_coefficient()
        #Finding the singular point to switch
        fc,hc = arrow.get_coordinates()
        maxi = len(trigausscode)
        singpt = (hc-1)%maxi
        trigausscode[hc],trigausscode[singpt] = trigausscode[singpt],trigausscode[hc]
        while trigausscode[0] != 'A':
            trigausscode = trigausscode[1:] + trigausscode[:1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
    else:
        if printerr:
            print('Error : cannot perform a negative head slide move')
        D2 = CoeffGaussDiagramTri()
        D2.set_coefficient(0)
    return D2

def positive_slide_move_foot_is_possible(D,arrow,printerr=False):
    '''A function that detects if an arrow can perform a positive slide move for its head'''
    trigausscode = D.get_trigausscode()
    namea = arrow.get_name()
    #Checking if -namea is close to 'A', 'B' or 'C'
    if distance_from_to(namea,'A',trigausscode) == 1 or distance_from_to(namea,'B',trigausscode) == 1 or distance_from_to(namea,'C',trigausscode) == 1:
        return True
    else:
        return False
    
def slide_move_positive_foot(D1,arrow,printerr=False):
    '''A function that performs a positive slide move for the head of an arrow'''
    #Input : D1 a SignedGaussDiagramWithOneArrow
    #Output : D2 a SignedGaussDiagramWithOneArrow obtained from performing a slide move with the head of the arrow on D1 or 0 if not possible
    #Sliding if possible
    if positive_slide_move_foot_is_possible(D1,arrow,printerr):
        trigausscode = deepcopy(D1.get_trigausscode())
        dtype = D1.get_type()
        homotopy = D1.get_homotopy()
        coeff = D1.get_coefficient()
        #Finding the singular point to switch
        fc,hc = arrow.get_coordinates()
        maxi = len(trigausscode)
        singpt = (fc+1)%maxi
        trigausscode[fc],trigausscode[singpt] = trigausscode[singpt],trigausscode[fc]
        while trigausscode[0] != 'A':
            trigausscode = trigausscode[1:] + trigausscode[:1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
    else:
        if printerr:
            print('Error : cannot perform a positive foot slide move')
        D2 = CoeffGaussDiagramTri()
        D2.set_coefficient(0)
    return D2

def negative_slide_move_foot_is_possible(D,arrow,printerr=False):
    '''A function that detects if an arrow can perform a positive slide move for its head'''
    trigausscode = D.get_trigausscode()
    namea = arrow.get_name()
    #Checking if -namea is close to 'A', 'B' or 'C'
    if distance_from_to('A',namea,trigausscode) == 1 or distance_from_to('B',namea,trigausscode) == 1 or distance_from_to('C',namea,trigausscode) == 1:
        return True
    else:
        return False

def slide_move_negative_foot(D1,arrow,printerr=False):
    '''A function that performs a positive slide move for the head of an arrow'''
    #Input : D1 a CoeffGaussDiagramWithOneArrow
    #Output : D2 a CoeffGaussDiagramWithOneArrow obtained from performing a slide move with the head of the arrow on D1 or 0 if not possible

    #Sliding if possible
    if negative_slide_move_foot_is_possible(D1,arrow,printerr):
        trigausscode = deepcopy(D1.get_trigausscode())
        dtype = D1.get_type()
        homotopy = D1.get_homotopy()
        coeff = D1.get_coefficient()
        #Finding the singular point to switch
        fc,hc = arrow.get_coordinates()
        maxi = len(trigausscode)
        singpt = (fc-1)%maxi
        trigausscode[fc],trigausscode[singpt] = trigausscode[singpt],trigausscode[fc]
        while trigausscode[0] != 'A':
            trigausscode = trigausscode[1:] + trigausscode[:1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
    else:
        if printerr:
            print('Error : cannot perform a negative foot slide move')
        D2 = CoeffGaussDiagramTri()
        D2.set_coefficient(0)
    return D2

def exists_in(E,D):
    '''A function that detects if a CoeffGaussDiagramTri D is in a LinearCombinationOfTriGaussDiagrams E'''
    #E = LinearCombinationOfGaussDiagrams, D = SignedGaussDiagramWithOneArrow
    exists = False
    for GD in E:
        if GD == D :
            exists = True
    return exists

def all_slide_move(D,printerr=False):
    '''A function that computes all slide moves from a diagram D'''
    E = LinearCombinationOfTriGaussDiagrams()
    if is_diagram_allowed(D):
        D1 = deepcopy(D)
        for arrow in D.get_arrows():
            D1 = deepcopy(D)
            if is_diagram_allowed(slide_move_positive_head(D1,arrow,printerr)) :
                D1 = slide_move_positive_head(D1,arrow,printerr)
                if not exists_in(E,D1):
                    E = E + D1
            D1 = deepcopy(D)
            if is_diagram_allowed(slide_move_positive_foot(D1,arrow,printerr)) :
                D1 = slide_move_positive_foot(D1,arrow,printerr)
                if not exists_in(E,D1):
                    E = E + D1
            D1 = deepcopy(D)
            if is_diagram_allowed(slide_move_negative_head(D1,arrow,printerr)) :
                D1 = slide_move_negative_head(D1,arrow,printerr)
                if not exists_in(E,D1):
                    E = E + D1
            D1 = deepcopy(D)
            if is_diagram_allowed(slide_move_negative_foot(D1,arrow,printerr)) :
                D1 = slide_move_negative_foot(D1,arrow,printerr)
                if not exists_in(E,D1):
                    E = E + D1
    else:
        if printerr :
            print('Not an allowed diagram')
    return E

Regular exchange move

In [ ]:
def exchange_move_is_possible(D,singular_arrow,regular_arrow):
    '''A function that detects if an arrow can perform a positive slide move for its head'''
    length = len(D.get_trigausscode())
    fa,ha = regular_arrow.get_coordinates()
    fs,hs = singular_arrow.get_coordinates()
    homa = regular_arrow.get_homotopy()
    homs = singular_arrow.get_homotopy()
    if dist(fa,fs,length) == 1 and dist(ha,hs,length) == 1 and homa == homs:
        #Almost every cases
        if (ha < hs and fa < fs) or (ha > hs and fa > fs):
            return True
        #Particular Case - Positive exchange with A as an endpoint
        elif (ha == length-1 and hs == 0 and fa < fs) or (fa == length-1 and fs == 0 and ha < hs):
            return True
        else:
            return False
    else:
        return False

def exchange_move(D,singular_arrow,regular_arrow,printerr=False):
    '''A function that operates an exchange_move between a singular_arrow and a regular_arrow in a CoeffTriGaussDiagram'''
    if exchange_move_is_possible(D,singular_arrow,regular_arrow):
        dtype = D.get_type()
        homotopy = D.get_homotopy()
        coeff = -D.get_coefficient()
        arrows = deepcopy(D.get_arrows())
        fa,ha = regular_arrow.get_coordinates()
        fs,hs = singular_arrow.get_coordinates()
        rarrow = deepcopy(regular_arrow)
        rname = rarrow.get_name()
        trigausscode = deepcopy(D.get_trigausscode())
        trigausscode[fa],trigausscode[fs] = trigausscode[fs],trigausscode[fa]
        trigausscode[ha],trigausscode[hs] = trigausscode[hs],trigausscode[ha]
        while trigausscode[0] != 'A':
            trigausscode = trigausscode[1:] + trigausscode[:1]
        D1 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
        return D1
    else:
        if printerr:
            print('Error : cannot perform an exchange move')
        return 0

    
def all_exchange_move(D,printerr=False):
    '''A function that computes all exchange moves from a diagram D'''
    E = LinearCombinationOfTriGaussDiagrams()
    d = D.get_d()
    hm = D.get_hm()
    ml = D.get_ml()
    arrows = D.get_arrows()
    if is_diagram_allowed(D):
        D1 = deepcopy(D)
        for a in arrows:
            E = E + exchange_move(D,d,a)
            E = E + exchange_move(D,hm,a)
            E = E + exchange_move(D,ml,a)
    else:
        if printerr :
            print('Not an allowed diagram')
    return E

Refined exchange move

In [ ]:
def exchange_move_is_possible(D,singular_arrow,regular_arrow):
    '''A function that detects if an arrow can perform a positive slide move for its head'''
    length = len(D.get_trigausscode())
    fa,ha = regular_arrow.get_coordinates()
    fs,hs = singular_arrow.get_coordinates()
    homa = regular_arrow.get_homotopy()
    homs = singular_arrow.get_homotopy()
    if dist(fa,fs,length) == 1 and dist(ha,hs,length) == 1 and homa == homs:
        #Almost every cases
        if (ha < hs and fa < fs) or (ha > hs and fa > fs):
            return True
        #Particular Case - Positive exchange with A as an endpoint
        elif (ha == length-1 and hs == 0 and fa < fs) or (fa == length-1 and fs == 0 and ha < hs):
            return True
        else:
            return False
    else:
        return False

def exchange_move(D,singular_arrow,regular_arrow,printerr=False):
    '''A function that operates an exchange_move between a singular_arrow and a regular_arrow in a CoeffTriGaussDiagram'''
    if exchange_move_is_possible(D,singular_arrow,regular_arrow):
        return singular_arrow.get_name()
    else:
        if printerr:
            print('Error : cannot perform an exchange move')
        return 0

    
def all_exchange_move(D,printerr=False):
    '''A function that computes all exchange moves from a diagram D'''
    E = LinearCombinationOfTriGaussDiagrams()
    d = D.get_d()
    hm = D.get_hm()
    ml = D.get_ml()
    arrows = D.get_arrows()
    res = []
    if is_diagram_allowed(D):
        D1 = deepcopy(D)
        for a in arrows:
            res.append(exchange_move(D,d,a))
            res.append(exchange_move(D,hm,a))
            res.append(exchange_move(D,ml,a))
    else:
        if printerr :
            print('Not an allowed diagram')
    return res

In [ ]:
def ends_are_switchable(end1,end2):
    '''A function that detects if end1 and end2 are switchable'''
    return abs(end1-end2) == 1

def do_cross(arrow1,arrow2):
    '''A function that computes if two arrows cross'''
    (f1,h1) = arrow1.get_coordinates()
    (f2,h2) = arrow2.get_coordinates()
    a = min(f1,h1)
    b = max(f1,h1)
    c = min(f2,h2)
    d = max(f2,h2)
    if (a < c < d < b) or ( c < d < a < b) or (c < a < b < d) or (a < b < c < d):
        return False
    else :
        return True

def insert_arrow(D,fa,ha,homa):
    '''A function that insert an arrow in a CoeffTriGaussDiagram'''
    #Modifying the trigausscode
    trigausscode = deepcopy(D.get_trigausscode())
    namea = len(D.get_arrows())+1
    if ha < fa :
        trigausscode.insert(ha,-namea)
        trigausscode.insert(fa,namea)
    else:
        trigausscode.insert(fa,namea)
        trigausscode.insert(ha,-namea)
    #Rolling the trigausscode until 'A' is in position 0
    while trigausscode[0] != 'A':
        trigausscode = trigausscode[1:] + trigausscode[:1]
    homotopy = deepcopy(D.get_homotopy())
    homotopy.append(homa)
    dtype = D.get_type()
    coeff = D.get_coefficient()
    return CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)

def compute_homotopy_for_3rd_slide_move_arrow(D,arrow1,arrow2):
    '''A function that computes the homotopy class of the 3rd arrow in a slide move for two arrows not in the triangle'''
    trigausscode = D.get_trigausscode()
    hom1 = border_homotopy_arrow(arrow1,trigausscode)
    hom2 = border_homotopy_arrow(arrow2,trigausscode)
    return Nbraid - (hom1 + hom2)

def left_right_slide_move_for_two_arrows(D,arrow1,arrow2,end1,end2):
    '''A functiont that returns the left and the right arrow of a slide move for two arrows not in the triangle'''
    #Going to the next endpoint of the two arrows from the singular point
    singpt = max(end1,end2)
    trigausscode = D.get_trigausscode()
    name1 = arrow1.get_name()
    name2 = arrow2.get_name()
    singpt = (singpt+1)%len(trigausscode)
    found = False
    i = singpt
    while found == False: 
    #for i in range(singpt,len(trigausscode)):
        if trigausscode[i] == -name1 or trigausscode[i] == name1:
            left = arrow1
            right = arrow2
            found = True
            break
        if trigausscode[i] == -name2 or trigausscode[i] == name2:
            left = arrow2
            right = arrow1
            found = True
            break 
        i = (i+1)%len(trigausscode)
    return left,right

def slide_move_for_two_arrows_is_doable_between(D,arrow1,arrow2,end1,end2,printerr=False):
    '''A function that detects if a slide move is doable between two arrows not in the triangle'''
    #Checking for error
    f1,h1 = arrow1.get_coordinates()
    f2,h2 = arrow2.get_coordinates()
    if (end1 != f1 and end1 != h1) or (end2 != f2 and end2 != h2):
        if printerr:
            print('Error : the ends do not correspond to the arrows :',arrow1.get_name(),arrow2.get_name())
        return False
    #If the ends are switchable, we compute the hypothetical RM3
    elif ends_are_switchable(end1,end2):
        #We compute the third arrow
        #Determining left and right
        leftarrow,rightarrow = left_right_slide_move_for_two_arrows(D,arrow1,arrow2,end1,end2)
        if printerr:
            print('Test du triangle pour Flèche de gauche : {}, Flèche de droite : {}'.format(leftarrow.get_name(),rightarrow.get_name()))
        fl,hl = leftarrow.get_coordinates()
        fr,hr = rightarrow.get_coordinates()
        List_coords = [coord for coord in (leftarrow.get_coordinates()+rightarrow.get_coordinates()) if (coord != end1 and coord != end2)]
        #Computing the coordinates of the third arrow
        #The head is the left arrow end
        if fl in List_coords:
            head = fl
        else:
            head = hl
        #The foot is the right arrow end
        if fr in List_coords:
            foot = fr
        else:
            foot = hr
        #Computing the coordinates depending on if the arrows cross
        #If the two arrows cross
        if do_cross(arrow1,arrow2):
            if head < foot :
                f3,h3 = foot + 1, head + 1
            else:
                f3,h3 = foot, head + 2
        else:
            if foot < head :
                f3,h3 = foot + 1, head + 1
            else:
                f3,h3 = foot + 2, head
        hom3 = compute_homotopy_for_3rd_slide_move_arrow(D,arrow1,arrow2)
        D2 = insert_arrow(D,f3,h3,hom3)
        if printerr:
            D2.plot()
            print('Plot de {} dans le cadre d\'un test de slide tri'.format(D2.get_trigausscode()))
        if is_diagram_allowed(D2,printerr):
            return True
        else:
            return False
        
def all_end_slide_move(D,arrow1,arrow2,printerr=False):
    (f1,h1) = arrow1.get_coordinates()
    (f2,h2) = arrow2.get_coordinates()
    dtype = D.get_type()
    homotopy = D.get_homotopy()
    coeff = D.get_coefficient()
    arrows = deepcopy(D.get_arrows())
    res = LinearCombinationOfTriGaussDiagrams()
    if slide_move_for_two_arrows_is_doable_between(D,arrow1,arrow2,f1,f2,printerr):
        pt1 = f1
        pt2 = f2
        #sliding
        trigausscode = deepcopy(D.get_trigausscode())
        trigausscode[pt1],trigausscode[pt2] = trigausscode[pt2],trigausscode[pt1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
        res = res + D2
    elif slide_move_for_two_arrows_is_doable_between(D,arrow1,arrow2,f1,h2,printerr):
        pt1 = f1
        pt2 = h2
        #sliding
        trigausscode = deepcopy(D.get_trigausscode())
        trigausscode[pt1],trigausscode[pt2] = trigausscode[pt2],trigausscode[pt1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
        res = res + D2
    elif slide_move_for_two_arrows_is_doable_between(D,arrow1,arrow2,h1,f2,printerr):
        pt1 = h1
        pt2 = f2
        #sliding
        trigausscode = deepcopy(D.get_trigausscode())
        trigausscode[pt1],trigausscode[pt2] = trigausscode[pt2],trigausscode[pt1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
        res = res + D2
    elif slide_move_for_two_arrows_is_doable_between(D,arrow1,arrow2,h1,h2,printerr):
        pt1 = h1
        pt2 = h2
        #sliding
        trigausscode = deepcopy(D.get_trigausscode())
        trigausscode[pt1],trigausscode[pt2] = trigausscode[pt2],trigausscode[pt1]
        D2 = CoeffGaussDiagramTri(dtype,trigausscode,homotopy,coeff)
        res = res + D2
    else:
        if printerr:
            print('Error : cannot perform a tri slide move')
    return res
    
def all_slide_move_for_two_arrows_not_in_triangle(D,printerr=False):
    '''A function that computes all slide moves for two arrows not in the triangle from a diagram D'''
    E = LinearCombinationOfTriGaussDiagrams()
    arrows = D.get_arrows()
    if is_diagram_allowed(D):
        D2 = deepcopy(D)
        for i in range(len(arrows)):
            for j in range(i+1,len(arrows)):
                arrow1 = arrows[i]
                arrow2 = arrows[j]
                E = E + all_end_slide_move(D,arrow1,arrow2,printerr)
    else:
        if printerr :
            print('Not an allowed diagram')
    return E

In [ ]:
def exists_in(E,D):
    '''A function that detects if a CoeffGaussDiagramTri D is in a LinearCombinationOfTriGaussDiagrams E'''
    #E = LinearCombinationOfGaussDiagrams, D = SignedGaussDiagramWithOneArrow
    exists = False
    if type(D) == str:
        return False
    else:
        for GD in E:
            if GD == D :
                exists = True
        return exists

def generate_invariant(D,printerr=False):
    '''A function that generates the invariant that contains a CoeffGaussDiagramTri D'''
    #D  = SignedGaussDiagramWithOneArrow 
    E = LinearCombinationOfTriGaussDiagrams()
    if is_diagram_allowed(D,printerr):
        E = E + D
        i = 0
        n = len(E)
        #i represents the diagram for which we compute all the possible moves
        while i<n :
            D = E[i]
            for Dj in all_slide_move(D,printerr):
                if not exists_in(E,Dj):
                    E = E + Dj
            for cor in all_exchange_move(D,printerr):
                if not exists_in(E,cor):
                    E = E + cor
            for Dj in all_slide_move_for_two_arrows_not_in_triangle(D,printerr):
                if not exists_in(E,Dj):
                    E = E + Dj
            i = i+1
            n = len(E)
        return E
        
    else:
        if printerr :
            print('Error : The diagram is not allowed. Will not compute invariant')
        return 'Error'

In [ ]:
def is_in_list(Liste,D):
    '''A function that detects if a CoeffGaussDiagramTri is in a List'''
    presence = False
    for G in Liste:
        if G.get_type() == D.get_type() and G.get_arrows() == D.get_arrows() and G.get_homotopy() == D.get_homotopy():
            presence = True
    return presence

def list_all_invariants(nbarrows,dtype=[1,1,'-'],maxi=50000,printerr=False):
    '''A function that list all invariants'''
    T = []
    #On commence par générer tous les diagrammes possibles :
    L = list_all_allowed_homogene_CoeffGaussDiagramTri(nbarrows,dtype,maxi)
    
    #Pour chaque diagramme dans la liste,on calcule son invariant
    while (len(L) > 0):
        D = L[0]
        E = generate_invariant(D,printerr)
        T.append(E)
        for G in E.get_expression():
            if is_in_list(L,G):
                H = deepcopy(G)
                H.set_coefficient(1)
                L.remove(H)
    return T

def list_all_type_invariants(nbarrows,maxi=50000,printerr=False):
    '''A function that list all invariants of all types'''
    T = []
    for dtype in ListType:
        T = T + list_all_invariants(nbarrows,dtype,maxi,printerr)
    return T

Generating the refined invariants

In [ ]:
T1 = list_all_type_invariants(1)
for i in range(len(T1)):
    name = '$W_{%d}^{1,aff}$'%i
    T1[i].set_name(name)
    T1[i].set_printname(True)
    T1[i].plot(i)
    filename = 'weight_aff_'+str(i)+'.eps'
    plt.savefig(filename, format='eps',bbox_inches='tight')

### I.c. Functions to compute the invariants

In [ ]:
def trigausscode_reconstruct_singular(arrows,size_tri,sign='+'):
    '''A function that computes a trigausscode from a set of arrows, the first arrows are d,hm,ml'''
    trigausscode =  l = [0] * (size_tri)
    for k in range(3,len(arrows)):
        a = arrows[k]
        if a.__class__.__name__ == "TriGaussArrow":
            i = a.get_name()
        elif a.__class__.__name__ == "TorusGaussArrow":
            i = a.get_number()
        else:
            return 'ERROR : Wrong class for the arrows'
        fa,ha = a.get_coordinates()
        trigausscode[fa] = i
        trigausscode[ha] = -i
    #Generating A,B,C
    fd,hd = arrows[0].get_coordinates()
    fhm,hhm = arrows[1].get_coordinates()
    right = fhm
    left = fd
    if sign == '-':
        right,left = left,right
    trigausscode[right] = 'A'
    trigausscode[hd] = 'B'
    trigausscode[left] = 'C'
    #Removing the remaining 0
    final_trigausscode = [el for el in trigausscode if el != 0]
    #Placing A first
    while final_trigausscode[0] != 'A':
        final_trigausscode = final_trigausscode[1:] + final_trigausscode[:1] 
    return final_trigausscode

def compute_temp_arrow(name,trigausscode):
    '''A function that computes the temporary coordinates for an arrow in a trigausscode'''
    for i in range(len(trigausscode)):
        if trigausscode[i] == name:
            fa = i
        elif trigausscode[i] == -name:
            ha = i
    return fa,ha

In [ ]:
def filtered_list(arrows,model_diagram,size_model_tri,diagram,size_tri,model_arrow,dictionary,sign,debug=False):
    '''A function that returns the arrows of diagram that are like model_arrow in model_diagram using a dictionary'''
    if debug:
        print('   DEBUT FILTRE')
        print('   Flèche modèle testée',model_arrow.get_name())
    if len(model_diagram) <= 0:
        return arrows
    else:
        #Initialization
        result_list = []
        #Constructing the model_trigausscode
        model_trigausscode = trigausscode_reconstruct_singular(model_diagram+[model_arrow],size_model_tri,sign)
        if debug:
            print('   Construction du modèle',model_trigausscode)
        model_f,model_h = compute_temp_arrow(model_arrow.get_name(),model_trigausscode)
        model_hom = model_arrow.get_homotopy()
        model_f_after = (model_f + 1)%len(model_trigausscode)
        model_f_before = (model_f - 1)%len(model_trigausscode)
        model_h_after = (model_h + 1)%len(model_trigausscode)
        model_h_before = (model_h - 1)%len(model_trigausscode)
        #Testing each arrow
        if debug:
            print('   Test des flèches par rapport à ',model_arrow.get_name())
        for arrow in arrows:
            if debug:
                print('        Test de la flèche',arrow.get_number())
            #Constructing a temporary dictionary
            temp_dictionary = deepcopy(dictionary)
            temp_dictionary[model_arrow.get_name()] = arrow.get_number()
            temp_dictionary[-model_arrow.get_name()] = -arrow.get_number()
            if debug:
                print('          ',temp_dictionary)
            #Constructing the trigausscode to be tested
            tested_trigausscode = trigausscode_reconstruct_singular(diagram+[arrow],size_tri,sign)
            if debug:
                print('          Construction du diagramme à tester',tested_trigausscode)
            fa,ha = compute_temp_arrow(arrow.get_number(),tested_trigausscode)
            #Testing position of the foot 
            f_after = (fa+1)%len(tested_trigausscode)
            f_before = (fa-1)%len(tested_trigausscode)
            test_arrow = tested_trigausscode[f_before] == temp_dictionary[model_trigausscode[model_f_before]] 
            test_arrow = test_arrow and (tested_trigausscode[f_after] == temp_dictionary[model_trigausscode[model_f_after]])
            #Testing position of the head
            h_after = (ha+1)%len(tested_trigausscode)
            h_before = (ha-1)%len(tested_trigausscode)
            test_arrow = test_arrow and (tested_trigausscode[h_before] == temp_dictionary[model_trigausscode[model_h_before]]) 
            test_arrow = test_arrow and (tested_trigausscode[h_after] == temp_dictionary[model_trigausscode[model_h_after]])
            #Testing the homotopy
            homa = arrow.get_homotopy()
            test_arrow = test_arrow and (homa == model_hom)
            if test_arrow:
                result_list.append(arrow)
            if debug:
                print('          ',arrow.get_number(),test_arrow)
        if debug:
            print('   FIN FILTRE')
        return result_list

In [ ]:
def test_type(D,GD):
    '''A function that returns the condition : the BraidData is of type GD'''
    if GD != 0:
        #Initialization
        D1 = deepcopy(D)
        #Getting the number of the three singular arrows
        clas = D1.get_Class()
        a,b,c = D1.get_RM3_a(),D1.get_RM3_b(),D1.get_RM3_c()
        d,hm,ml = name_arrows(a,b,c)
        #Testing if we are in the appropriate RM3 type
        homd = GD.get_d().get_homotopy()
        homhm = GD.get_hm().get_homotopy()
        homml = GD.get_ml().get_homotopy()
        return homd == d.get_homotopy() and homhm == hm.get_homotopy() and homml == ml.get_homotopy()
    else:
        return False

def contribution(I0,G,debug=False):
    '''A function that returns the contribution of a BraidData G with respect to a CoefficientGaussDiagramTri I0'''
    #Testing the type
    if test_type(G,I0):
        #Creating a dictionary that will translate the TriGaussArrows of I0 into TorusGaussArrows for G
        current_dictionary = {}
        arrow_d,arrow_hm,arrow_ml = name_arrows(G.get_RM3_a(),G.get_RM3_b(),G.get_RM3_c())
        d = arrow_d.get_number()
        hm = arrow_hm.get_number()
        ml = arrow_ml.get_number()
        dtype = I0.get_type()
        sign = dtype[2]
        A = [-ml,hm]
        B = [-hm,-d]
        C = [d,ml]
        if sign == '-':
            B,C = C,B
        current_dictionary["A"] = "A"
        current_dictionary["B"] = "B"
        current_dictionary["C"] = "C"
        #Initialisation of the TriGaussDiagram to test
        current_model_diagram = [I0.get_d(),I0.get_hm(),I0.get_ml()]
        size_model_tri = len(I0.get_trigausscode())
        #Initialisation of the Arrows that are tested in G
        current_diagram = [arrow_d,arrow_hm,arrow_ml]
        size_tri = len(G.get_TorusGaussCode().get_GaussDiagram())
        #Initialisation of the index of the model arrow in I0
        current_arrow_index = 0
        #Getting the number of arrows in I0
        n = len(I0)
        #Initialisation of the list of corresponding set of arrows
        corresponding_arrows_set_list = []
        #Defining a function that will recursively compute the arrows that correspond
        def recursive_test(current_model_diagram,current_diagram,current_arrow_index,current_dictionary):
            if current_arrow_index == n:
                corresponding_arrows_set_list.append(current_diagram)
            else:
                model_arrow = I0.get_arrows()[current_arrow_index]
                if debug:
                    print('Test pour ',model_arrow.get_name())
                remaining_arrows = [arrow for arrow in G.get_TorusGaussCode().get_Arrows() if arrow not in current_diagram]
                if debug:
                    print('Remaining arrows',[arrow.get_number() for arrow in remaining_arrows])
                #For each arrow that is like model_arrow, we test the other arrows of model_diagram
                for arrow in filtered_list(remaining_arrows,current_model_diagram,size_model_tri,current_diagram,size_tri,model_arrow,current_dictionary,sign,debug):
                    #Updating to test the other arrows of I0
                    new_model_diagram = current_model_diagram + [model_arrow]
                    new_diagram = current_diagram + [arrow]
                    new_arrow_index = current_arrow_index + 1
                    new_dictionary = deepcopy(current_dictionary)
                    new_dictionary[model_arrow.get_name()] = arrow.get_number()
                    new_dictionary[-model_arrow.get_name()] = -arrow.get_number()
                    recursive_test(new_model_diagram,new_diagram,new_arrow_index,new_dictionary)
        recursive_test(current_model_diagram,current_diagram,current_arrow_index,current_dictionary)
        #Now corresponding_arrows_set_list contains all the set of arrows that are like the model_diagram I0
        res = 0
        coeff = I0.get_coefficient()
        if debug:
            print('Coefficient',coeff)
        if debug and len(corresponding_arrows_set_list) > 0 :
            print('Set of corresponding arrows : ')
            for index in range(len(corresponding_arrows_set_list)):
                print('   Set ',index,[arrow.get_number() for arrow in corresponding_arrows_set_list[index]])
        #We get the contribution of each set
        for arrows_set in corresponding_arrows_set_list:
            temp_res = 1
            for i in range(3,len(arrows_set)):
                temp_res = temp_res*arrows_set[i].get_writhe()
            res = res + temp_res
        return res
    else:
        return 0
    
def invariant_compute(D,List_Expression,debug=False):
    '''A function that computes the invariants given by List_Expression for D'''
    D1 = deepcopy(D)
    List_Expressions = deepcopy(List_Expression)
    res = []
    for LC in List_Expressions:
        I = 0
        if LC != 0 :
            #Part with Gauss Diagram
            for GD in LC.get_expression():
                coeff = GD.get_coefficient()
                cont = contribution(GD,D1)
                if debug and cont != 0:
                    contribution(GD,D1,debug)
                    print('Contribution : ',coeff*cont)
                    plt.figure()
                    GD.plot()
                I = I + coeff*cont
            #Part with correction term
            if LC.get_correction() != []:
                a,b,c = D1.get_RM3_a(),D1.get_RM3_b(),D1.get_RM3_c()
                d,hm,ml = name_arrows(a,b,c)
                wd = d.get_writhe()
                whm = hm.get_writhe()
                wml = ml.get_writhe()
                for cor in LC.get_correction():
                    if cor == 'd':
                        I = I + (1/2)*(wd-1)
                    if cor == 'hm':
                        I = I + (1/2)*(whm-1)
                    if cor == 'ml':
                        I = I + (1/2)*(wml-1)
        res.append(I)
    return array(res)

Importing the space of laurent multivariable polynomials (add as much as needed)

In [ ]:
R.<x0,x1,x2,x3,x4,x5> = LaurentPolynomialRing(QQ)
x = [x0,x1,x2,x3,x4,x5]

T1raf is the variable that have all the weights

In [ ]:
T1raf = [[T1[i]] for i in range(len(T1))]

T1mult is the variable that have the weights for the multivariable invariants

In [ ]:
T1mult = [[T1[index] for index in range(j,6+j)] for j in range(6)]

Pushing $Delta^2$ in order to apply $rot$

In [ ]:
def push_delta(D,List_invariants,i,debug=False):
    '''A function that pushes Delta through a generator and computes the contribution with respect to a list of invariants'''
    #D is a BraidData, i is the position of the generator
    #Getting a copy of D
    D1 = deepcopy(D)
    diag = []
    singular = []
    cont = [0 for index in range(len(List_invariants))]
    GD = [List_invariants[index][0][0] for index in range(len(List_invariants))]
    #Getting the braid element in position i
    ei = D.get_BraidClosure()[i]
    #Pushing through 1 : 1 123121
    if ei == 1 :
        #Commutation move : 1123121 = 1121321
        D1.CommutationMove(i+3)
        diag.append(deepcopy(D1))
        #Reidemeister move III : 1121321 --> 1212321 
        d = D1.ReidemeisterMoveIII(i+1,i+2,i+3)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Reidemeister move III : 1212321 --> 1213231
        d = D1.ReidemeisterMoveIII(i+3,i+4,i+5)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Commutation move : 1213231 = 1231231
        D1.CommutationMove(i+2)
        diag.append(deepcopy(D1))
        #Commutation move : 1231231 = 1231213 ====> 123121 3
        D1.CommutationMove(i+5)
        diag.append(deepcopy(D1))
        
    #Pushing through -1 : -1 123121
    elif ei == -1 :
        #Reidemeister move II : -1123121 --> 23121
        D1.ReidemeisterMoveII_delete(i)
        diag.append(deepcopy(D1))
        #Reidemeister move II : 23121 --> 1-123121
        D1.ReidemeisterMoveII_create(1,i)
        diag.append(deepcopy(D1))
        #Commutation move : 1-123121 = 1-121321
        D1.CommutationMove(i+3)
        diag.append(deepcopy(D1))
        #Reidemeister move III : 1-121321 -->121-2321
        d = D1.ReidemeisterMoveIII(i+1,i+2,i+3)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Reidemeister move III : 121-2321 --> 12132-31
        d = D1.ReidemeisterMoveIII(i+3,i+4,i+5)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Commutation move : 12132-31 = 12312-31
        D1.CommutationMove(i+2)
        diag.append(deepcopy(D1))
        #Commutation move : 12312-31 = 123121-3 ====> 123121 -3
        D1.CommutationMove(i+5)
        diag.append(deepcopy(D1))
        
    #Pushing through 2 : 2 123121
    elif ei == 2 :
        #Reidemeister move III : 2123121 --> 1213121
        d = D1.ReidemeisterMoveIII(i,i+1,i+2)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Commutation move : 1213121 = 1231121
        D1.CommutationMove(i+2)
        diag.append(deepcopy(D1))
        #Reidemeister move III : 1231121 --> 1231212 ====> 123121 2
        d = D1.ReidemeisterMoveIII(i+4,i+5,i+6)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
    
    #Pushing through -2 : -2 123121
    elif ei == -2 :
        #Reidemeister move III : -2123121 --> 12-13121
        d = D1.ReidemeisterMoveIII(i,i+1,i+2)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Commutation move : 12-13121 = 123-1121
        D1.CommutationMove(i+2)
        diag.append(deepcopy(D1))
        #Reidemeister move II : 123-1121 --> 12321
        D1.ReidemeisterMoveII_delete(i+3)
        diag.append(deepcopy(D1))
        #Reidemeister move II : 12321 --> 1231-121
        D1.ReidemeisterMoveII_create(1,i+3)   
        diag.append(deepcopy(D1))
        #Reidemeister move III : 1231-121 --> 123121-2 ====> 123121 -2
        d = D1.ReidemeisterMoveIII(i+4,i+5,i+6)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        
    #Pushing through 3 : 3 123121
    elif ei == 3 :
        #Commutation move : 3123121 = 1323121
        D1.CommutationMove(i)
        diag.append(deepcopy(D1))
        #Reidemeister move III : 1323121 --> 1232121
        d = D1.ReidemeisterMoveIII(i+1,i+2,i+3)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Reidemeister move III : 1232121 --> 1231211 ====> 123121 1
        d = D1.ReidemeisterMoveIII(i+3,i+4,i+5)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        
    #Pushing through -3 : -3 123121
    elif ei == -3 :
        #Commutation move : -3123121 = 1-323121
        D1.CommutationMove(i)
        diag.append(deepcopy(D1))
        #Reidemeister move III : 1-323121 --> 123-2121
        d = D1.ReidemeisterMoveIII(i+1,i+2,i+3)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Reidemeister move III : 123-2121 --> 12312-11
        d = D1.ReidemeisterMoveIII(i+3,i+4,i+5)
        I = [0 for index in range(len(List_invariants))]
        for index in range(len(I)):
            I[index] = invariant_compute(d,List_invariants[index])
        s = d.get_sign()
        for index in range(len(List_invariants)):
            if test_type(d,GD[index]):
                tempcont = 1
                for jindex in range(len(List_invariants[index])):
                    tempcont = tempcont*x[jindex]**I[index][jindex]
                cont[index] = cont[index] + s*tempcont
                if debug:
                    print(d.get_Class())
                    plt.figure()
                    d.plot_diagram()
                    print(s*tempcont)
                    print(invariant_compute(d,List_invariants[index],debug))
        diag.append(d)
        diag.append(deepcopy(D1))
        singular.append(d)
        #Reidemeister move II : 12312-11 --> 12312
        D1.ReidemeisterMoveII_delete(i+5)
        diag.append(deepcopy(D1))
        #Reidemeister move II : 12312 --> 123121-1 ====> 123121 -1
        D1.ReidemeisterMoveII_create(1,i+5)
        diag.append(deepcopy(D1))
    return D1,diag,singular,array(cont)

def add_delta(D,i):
    '''A function that adds Delta and Delta^-1 to a braid closure at position i'''
    #In : G,S torus gauss code, braid closure
    #Out : H,R torus gauss code, braid closure
    D1 = deepcopy(D)
    diag = []
    #Adding Delta and Delta^-1
    d = D1.ReidemeisterMoveII_create(1,i)
    diag.append(d)
    d = D1.ReidemeisterMoveII_create(2,i+1)
    diag.append(d)
    d = D1.ReidemeisterMoveII_create(3,i+2)
    diag.append(d)
    d = D1.ReidemeisterMoveII_create(1,i+3)
    diag.append(d)
    d = D1.ReidemeisterMoveII_create(2,i+4)
    diag.append(d)
    d = D1.ReidemeisterMoveII_create(1,i+5)
    diag.append(d)
    
    return D1,diag

def rot(D,List_invariants,debug=False):
    '''A function that operates the loop rot(S) on a braid'''
    #In : S braid closure
    #Out : H,R torus gauss code, braid closure
    D1 = deepcopy(D)
    n = D1.get_length()
    Inv = [int(0) for index in range(len(List_invariants))]
    #Case S = S'DD
    R = D1.get_BraidClosure()
    rot = [deepcopy(D1)]
    rot_sing = []
    if n >= 12: 
        if (R[n-12],R[n-11],R[n-10],R[n-9],R[n-8],R[n-7],R[n-6],R[n-5],R[n-4],R[n-3],R[n-2],R[n-1]) == (1,2,3,1,2,1,1,2,3,1,2,1):
            #m is the size of S'
            m = n-12
            #Applying Delta once :
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m-i-1,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
            #Applying Delta a second time :
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m+5-i,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
        elif (R[n-6],R[n-5],R[n-4],R[n-3],R[n-2],R[n-1]) == (1,2,3,1,2,1):
            #Size of S'
            m = n-6
            #Adding Delta once
            D1,diag = add_delta(D1,n)
            rot.extend(diag)
            #Applying Delta once : 
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m-i-1,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
            #Applying Delta a second time :
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m+5-i,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
        else:
            m = n
            #Adding Delta twice
            D1,diag = add_delta(D1,n)
            rot.extend(diag)
            D1,diag = add_delta(D1,n+6)
            rot.extend(diag)
            #Applying Delta once : 
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m-i-1,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
            #Applying Delta a second time :
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m+5-i,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
            
    elif n>= 6: 
        if (R[n-6],R[n-5],R[n-4],R[n-3],R[n-2],R[n-1]) == (1,2,3,1,2,1):
            #Size of S'
            m = n-6
            #Adding Delta once
            D1,diag = add_delta(D1,n)
            rot.extend(diag)
            #Applying Delta once : 
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m-i-1,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
            #Applying Delta a second time :
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m+5-i,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
        else:
            m = n
            #Adding Delta twice
            D1,diag = add_delta(D1,n)
            rot.extend(diag)
            D1,diag = add_delta(D1,n+6)
            rot.extend(diag)
            #Applying Delta once : 
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m-i-1,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
            #Applying Delta a second time :
            for i in range(m):
                D1,diag,sing,cont = push_delta(D1,List_invariants,m+5-i,debug)
                Inv = Inv + cont
                rot.extend(diag)
                rot_sing.extend(sing)
    else:
        m = n
        #Adding Delta twice
        D1,diag = add_delta(D1,n)
        rot.extend(diag)
        D1,diag = add_delta(D1,n+6)
        rot.extend(diag)
        #Applying Delta once : 
        for i in range(m):
            D1,diag,sing,cont = push_delta(D1,List_invariants,m-i-1,debug)
            Inv = Inv + cont
            rot.extend(diag)
            rot_sing.extend(sing)
        #Applying Delta a second time :
        for i in range(m):
            D1,diag,sing,cont = push_delta(D1,List_invariants,m+5-i,debug)
            Inv = Inv + cont
            rot.extend(diag)
            rot_sing.extend(sing)
    return rot,rot_sing,Inv

def List_sigmas(S,List_invariants,debug=False):
    D = BraidData(S)
    roti,rot_sing,Inv = rot(D,List_invariants,debug)
    return Inv

## II - Examples and results

In [ ]:
S = [1,2,3]
print('Multivariable invariants : {}'.format(list(List_sigmas(S,T1mult))))
print('Refined invariants : {}'.format(list(List_sigmas(S,T1raf))))

In [ ]:
S = [2, -1, 2, -3, 2, 3, 2, 3, 3]
print('Multivariable invariants : {}'.format(list(List_sigmas(S,T1mult))))
print('Refined invariants : {}'.format(list(List_sigmas(S,T1raf))))

In [ ]:
S = [2, 3, -1, -2, 3, 3, 2, 3, 3]
print('Multivariable invariants : {}'.format(list(List_sigmas(S,T1mult))))
print('Refined invariants : {}'.format(list(List_sigmas(S,T1raf))))

### II.a. Functions to generate random braids

In [ ]:
import random

def random_braid(size=3):
    '''A function that generates a random 4-braid that is a knot'''
    isknot = False
    is4braid = False
    #We generate a random 4-braid until it is a knot
    while isknot == False or is4braid == False:
        elem = random.choice([-1,1,-2,2,-3,3])
        braid = [elem]
        #Generating a random braid
        while len(braid) < size:
            elem = random.choice([-1,1,-2,2,-3,3])
            if elem != -braid[-1]:
                braid.append(elem)
        is4braid = min(abs(array(braid))) == 1 and max(abs(array(braid))) == 3
        L = Link(B(braid))
        isknot = L.is_knot()
    return braid

def random_positive_braid(size=3):
    '''A function that generates a random 4-braid that is a knot'''
    isknot = False
    is4braid = False
    #We generate a random 4-braid until it is a knot
    while isknot == False or is4braid == False:
        elem = random.choice([1,2,3])
        braid = [elem]
        #Generating a random braid
        while len(braid) < size:
            elem = random.choice([1,2,3])
            if elem != -braid[-1]:
                braid.append(elem)
        is4braid = min(abs(array(braid))) == 1 and max(abs(array(braid))) == 3
        L = Link(B(braid))
        isknot = L.is_knot()
    return braid

### II.b. Looking for non invertible braids

Functions to look for non invertible braids using the 1-cocycles (uncomment if needed)

In [ ]:
# def reverse_braid(S):
#     '''A function that returns the reverse braid of S'''
#     return [s for s in reversed(S)]

# def find_non_invertible_braid(braidlength=21,nbattempts=30):
#     '''A function that find a non invertible braid'''
#     attempt = 1
#     found = False
#     while found == False and attempt <= nbattempts:
#         S = random_braid(braidlength)
#         InvS = List_sigmas(S,T1raf)
#         revS = reverse_braid(S)
#         InvrevS = List_sigmas(revS,T1raf)
#         print('Attempt n°{} : {},{}'.format(attempt,S,list(InvS)==list(InvrevS)))
#         print('   {}'.format(InvS))
#         if list(InvS) != list(InvrevS) :
#             found = True
#         if attempt%10 == 0:
#             print('{} tries'.format(attempt))
#         attempt = attempt + 1
#     if found:
#         print('Found one !!!!!!!!!!')
#         print('Braid : ',S)
#         print(InvS)
#         print('Inverted braid : ',revS)
#         print(InvrevS)
#     return 'OK'

In [ ]:
# find_non_invertible_braid(11,50)

### II.c. Invariants dimension

Functions to estimate the linear dependance of the invariants. Uncomment if needed

In [ ]:
# def estimation_dimension(repeat=20,maxi=15,print_test=False):
#     dim = 0
#     if print_test:
#         print('Start of the computations')
#     #Repeating the process repeat times
#     for k in range(repeat):
#         if print_test:
#             print('   Process number {}'.format(k+1))
#             print('      Generating the matrix')
#         M = []
#         braid = []
#         #Generating as much braids as index in T)
#         for j in range(len(T1raf)):
#             #Generating a random size
#             size = 0
#             n2 = size%2
#             while (n2 == 0):
#                 size = int(round(random.random()*maxi))
#                 n2 = size%2
#             #Generating a random braid of size size
#             S = random_braid(size)
#             if print_test:
#                 print('         Braid : {}'.format(S))
#             #Computing the different Invariants for S 
#             InvS = List_sigmas(S,T1raf)
#             Lis = []
#             for i in range(len(T1raf)):
#                 Lis.append(InvS[i])
#             if print_test:
#                 print('         Invariants : {}'.format(Lis))
#             #Storing the different datas
#             braid.append(S)
#             M.append(Lis)
#             if print_test:
#                 print('         Updating the matrices')
#                 print('         Braids matrix : {}'.format(braid))
#                 print('         Invariants matrix : {}'.format(M))
#         #Computing the matrix rank
#         M = array(M)
#         dimM = linalg.matrix_rank(M)
#         dim = max(dim,dimM)
#         if print_test:
#             print('      Final invariants matrix : {}'.format(M))
#             print('      Matrix rank : {}'.format(dimM))
#             print('      Dimension : {}'.format(dim))
#     return dim

In [ ]:
# estimation_dimension(11,5,True)

### II.d. Functions to compare the invariants with finite type invariants defined by Gauss formulas

Uncomment if needed

In [ ]:
# def circle(center, radius,color):
#     ax = plt.axes()
#     theta = linspace(0, 2*pi, 100)
#     x1 = center[0]+radius*cos(theta)
#     x2 = center[1]+radius*sin(theta)
#     plt.plot(x1,x2,color)
#     plt.axis('scaled')
#     plt.axis('off')
#     return 0

# def compute_color(homotopy):
#     if homotopy == 1:
#         return 'red'
#     elif homotopy == 2:
#         return 'blue'
#     elif homotopy == 3:
#         return 'green'
#     else:
#         return 'black'

# class GaussArrow:
#     "An arrow in a Gauss diagram "
#     def __init__(self,foot=-1,head=-1,homotopy=0):
#         self.__foot = foot
#         self.__head = head
#         self.__homotopy = homotopy
#         self.__color = compute_color(homotopy)
#     def set_foot(self,f):
#         self.__foot = f
#     def set_head(self,h):
#         self.__head = h
#     def set_homotopy(self,h):
#         self.__homotopy = h
#         self.__color = compute_color(h)
#     def get_foot(self):
#         return self.__foot
#     def get_head(self):
#         return self.__head
#     def get_homotopy(self):
#         return self.__homotopy
#     def get_color(self):
#         return self.__color
#     def get_infos(self):
#         print( 'Foot coordinate :',self.__foot)
#         print( 'Head coordinate :',self.__head)
#         print( 'Homotopy class:',self.__homotopy)
#         print( 'Color :',self.__color)
#     def plot_arrow(self,center=[0,0],nbarrows=1):
#         #Computing the arrow coordinates in the circle
#         x = center[0]
#         y = center[1]
#         fangle = self.get_foot()*(2*pi/(2*nbarrows)) + pi/(2*nbarrows)
#         hangle = self.get_head()*(2*pi/(2*nbarrows)) + pi/(2*nbarrows)
#         foot = (x + cos(fangle),y + sin(fangle))
#         head = (x + cos(hangle),y + sin(hangle))
#         color = self.get_color()
#         #Plotting the arrow
#         plt.annotate('', xy = head, xytext  = foot,
#                         arrowprops = {'color': color,'arrowstyle' : '->'})
#     def __eq__(self,A):
#         if A.__class__.__name__ == 'GaussArrow':
#             if self.get_foot() == A.get_foot() and self.get_head() == A.get_head() and self.get_homotopy() == A.get_homotopy():
#                 return True
#             else:
#                 return False
#         else:
#             return False
# class GaussDiagram:
#     "A circle together with marked arrows"
#     def __init__(self,arrows=[]):
#         self.__arrowslist = arrows
#     def set_arrowslist(self,arrows):
#         self.__arrowslist = arrows
#     def get_arrowslist(self):
#         return self.__arrowslist
#     def add_arrow(self,a):
#         self.__arrowslist.append(a)
#     def del_arrow(self,i):
#         '''A method that the arrow that is in position i in the arrowslist'''
#         self.__arrowslist.pop(i)
#     def plot_diagram(self,center=[0,0]):
#         #Plotting the circle
#         x = center[0]
#         y = center[1]
#         circle((x,y),1,'black')
#         plt.axis('off')
#         #Plotting the arrows
#         nbarrows = len(self.get_arrowslist())
#         for arrow in self.get_arrowslist():
#             arrow.plot_arrow(center,nbarrows)
#     def __eq__(self,G):
#         if G.__class__.__name__ == 'GaussDiagram':
#             if self.get_arrowslist() == G.get_arrowslist():
#                 return True
#             else:
#                 return False
#         else:
#             return False
        
# class GaussDiagramWithCoefficient(GaussDiagram):
#     "A Gauss Diagram with Coefficient"
#     def __init__(self,arrows=[],c=1):
#         GaussDiagram.__init__(self,arrows)
#         self.__coefficient = c
#     def set_coefficient(self,coeff):
#         self.__coefficient = coeff
#     def get_coefficient(self):
#         return self.__coefficient
#     def __mul__(self,n):
#         G = deepcopy(self)
#         G.set_coefficient(self.get_coefficient()*n)
#         return G
#     def __rmul__(self,n):
#         return self*n
#     def __eq__(self,G):
#         if G.__class__.__name__ == 'GaussDiagramWithCoefficient':
#             if G != 0 and self.get_arrowslist() == G.get_arrowslist():
#                 return True
#             elif G == 0 and self.get_coefficient() == 0:
#                 return True
#             else:
#                 return False
#         else:
#             return False
#     def plot_diagram(self,center=[0,0]):
#         #Plotting the circle
#         x = center[0]
#         y = center[1]
#         circle((x,y),1,'black')
#         plt.axis('off')
#         #Plotting the arrows
#         nbarrows = len(self.get_arrowslist())
#         for arrow in self.get_arrowslist():
#             arrow.plot_arrow(center,nbarrows)
#         #Plotting the coefficient
#         x = center[0] - 2 - 0.2
#         y = center[1] - 0.1
#         coeff = self.get_coefficient()
#         if coeff > 1:
#             coeff = '+ ' + str(coeff)
#         elif coeff == 1:
#             coeff = '+'
#         elif coeff == -1:
#             coeff = '-'
#         plt.text(x,y,coeff,color='black',size='xx-large')
    
# class LinearCombinationOfGaussDiagrams:
#     "A linear combination of Gauss diagrams"
#     def __init__(self,GD=[]):
#         self.__name = 'INIT'
#         if GD.__class__.__name__ == 'GaussDiagramWithCoefficient' :
#             self.__expression = [GD]
#         elif GD != []:
#             D = GaussDiagramWithCoefficient(GD)
#             self.__expression = [D]
#         else:
#             self.__expression = []
#     def set_name(self,value):
#         self.__name = value
#     def get_expression(self):
#         return self.__expression
#     def get_name(self):
#         return self.__name
#     def update(self):
#         Lis = []
#         for GD in self:
#             if GD.get_coefficient() == 0:
#                 Lis.append(GD)
#         for GD in Lis:
#             self.get_expression().remove(GD)
#     def __len__(self):
#         return len(self.__expression)
#     def __getitem__(self, key):
#         return self.get_expression()[key]
#     def __setitem__(self, key, value):
#         self.__expression[key] = value
#     def __delitem__(self, key):
#         self.__expression.pop(key)
#     def __missing__(self, key):
#         print( '%i is not a valid index'% key)
#     def __iter__(self):
#         return iter(self.__expression)
#     def __reversed__(self):
#         return reversed(self.__expression)
#     def __contains__(self, item):
#         ans = False
#         for GD in self:
#             if GD == item:
#                 ans = True
#         return ans
#     def __mul__(self,n):
#         E = deepcopy(self)
#         for i in range(len(self)):
#             E[i] = E[i]*n
#         #E.update()
#         return E
#     def __rmul__(self,n):
#         return self*n
#     def __add__(self, T):
#         E = deepcopy(self)
#         if T.__class__.__name__ == 'LinearCombinationOfGaussDiagrams':
#             for GD in T:
#                 boolin = False
#                 for i in range(len(self)):
#                     if E[i] == GD:
#                         E[i].set_coefficient(E[i].get_coefficient() + GD.get_coefficient())
#                         boolin = True
#                 if boolin == False:
#                     E.get_expression().append(GD)
#         elif T.__class__.__name__ == 'GaussDiagramWithCoefficient':
#             boolin = False
#             for i in range(len(self)):
#                 if E[i] == T:
#                     E[i].set_coefficient(E[i].get_coefficient() + T.get_coefficient())
#                     boolin = True
#             if boolin == False:
#                 E.get_expression().append(T)
#         return E
#     def __neg__(self):
#         return (-1)*self
#     def __sub__(self,GD):
#         return self + (-GD)
#     def plot_expression(self,m=0):
#         fig = plt.figure(m, figsize=(6.4*len(self), 7))
#         #Plotting the name
#         if self.get_name() != 'INIT':
#             x = - 0.2 - 4
#             y = - 0.1
#             plt.text(x,y,self.get_name(),color='black',size='xx-large')
#             plt.text(x+1,y,'=',color='black',size='xx-large')
#         if len(self) <= 5 :
#             plt.xlim(-2,len(self)*4)
#             plt.ylim(-1.2,1.2)
#             center=array([-4,0])
#             for GD in self:
#                 center = center + array([4,0])
#                 GD.plot_diagram(center)
#         elif len(self) <= 10:
#             plt.xlim(-2,20)
#             plt.ylim(-3.7,1.2)
#             center=array([-4,0])
#             for i in range(5):
#                 GD = self[i]
#                 center = center + array([4,0])
#                 GD.plot_diagram(center)
#             center=array([-4,-2.5])
#             for i in range(5,len(self)):
#                 GD = self[i]
#                 center = center + array([4,0])
#                 GD.plot_diagram(center)
#         else:
#             plt.xlim(-2,20)
#             plt.ylim(-6.2,1.2)
#             center=array([-4,0])
#             for i in range(5):
#                 GD = self[i]
#                 center = center + array([4,0])
#                 GD.plot_diagram(center)
#             center=array([-4,-2.5])
#             for i in range(5,10):
#                 GD = self[i]
#                 center = center + array([4,0])
#                 GD.plot_diagram(center)
#             center=array([-4,-5])
#             for i in range(10,len(self)):
#                 GD = self[i]
#                 center = center + array([4,0])
#                 GD.plot_diagram(center)
                
# #1
# a = GaussArrow(2,1,1)
# b = GaussArrow(3,0,2)
# D1 = LinearCombinationOfGaussDiagrams([a,b])
# #2
# c = GaussArrow(3,0,3)
# D2 = LinearCombinationOfGaussDiagrams([a,c])
# #3
# d = GaussArrow(2,1,2)
# D3 = LinearCombinationOfGaussDiagrams([c,d])
# #4
# e = GaussArrow(2,0,1)
# f = GaussArrow(3,1,3)
# D4 = LinearCombinationOfGaussDiagrams([e,f])
# #5
# g = GaussArrow(3,1,2)
# D5 = LinearCombinationOfGaussDiagrams([e,g])
# #6
# h = GaussArrow(3,1,1)
# k = GaussArrow(2,0,2)
# D6 = LinearCombinationOfGaussDiagrams([h,k])
# #7
# D7 = LinearCombinationOfGaussDiagrams([f,k])
# #8
# l = GaussArrow(2,0,3)
# D8 = LinearCombinationOfGaussDiagrams([l,h])
# #9
# D9 = LinearCombinationOfGaussDiagrams([g,l])
# #10
# m = GaussArrow(0,3,2)
# D10 = LinearCombinationOfGaussDiagrams([a,m])
# #11
# o = GaussArrow(0,3,1)
# D11 = LinearCombinationOfGaussDiagrams([a,o])
# #12
# p = GaussArrow(1,2,3)
# D12 = LinearCombinationOfGaussDiagrams([p,b])
# #13
# D13 = LinearCombinationOfGaussDiagrams([p,c])

# def plot_invariant(l2,l3,l4,l5,l6,l8,l10,l12,name,m=0):
#     l1 = -l3+l4+l12-2*l2+l8+l10
#     l7 = l6+l3+l2-l8-l10
#     l9 = l3-l4+l2-l10+l5
#     l11 = -l6-2*l3+2*l4+2*l12-4*l2+2*l8+2*l10-l5
#     l13 = -l6+l4-2*l2+l8+2*l10-l5
#     E = l1*D1+l2*D2+l3*D3+l4*D4+l5*D5+l6*D6+l7*D7+l8*D8+l9*D9+l10*D10+l11*D11+l12*D12+l13*D13
#     E.update()
#     E.set_name(name)
#     E.plot_expression(m)
    
# def generate_finite_invariant(l2,l3,l4,l5,l6,l8,l10,l12,name):
#     l1 = -l3+l4+l12-2*l2+l8+l10
#     l7 = l6+l3+l2-l8-l10
#     l9 = l3-l4+l2-l10+l5
#     l11 = -l6-2*l3+2*l4+2*l12-4*l2+2*l8+2*l10-l5
#     l13 = -l6+l4-2*l2+l8+2*l10-l5
#     E = l1*D1+l2*D2+l3*D3+l4*D4+l5*D5+l6*D6+l7*D7+l8*D8+l9*D9+l10*D10+l11*D11+l12*D12+l13*D13
#     E.update()
#     E.set_name(name)
#     return E

# Tf = []
# Tf.append(generate_finite_invariant(1,0,0,0,0,0,0,0,'$I_{%d}$'%0))
# Tf.append(generate_finite_invariant(0,1,0,0,0,0,0,0,'$I_{%d}$'%1))
# Tf.append(generate_finite_invariant(0,0,1,0,0,0,0,0,'$I_{%d}$'%2))
# Tf.append(generate_finite_invariant(0,0,0,1,0,0,0,0,'$I_{%d}$'%3))
# Tf.append(generate_finite_invariant(0,0,0,0,1,0,0,0,'$I_{%d}$'%4))
# Tf.append(generate_finite_invariant(0,0,0,0,0,1,0,0,'$I_{%d}$'%5))
# Tf.append(generate_finite_invariant(0,0,0,0,0,0,1,0,'$I_{%d}$'%6))
# Tf.append(generate_finite_invariant(0,0,0,0,0,0,0,1,'$I_{%d}$'%7))

# def same_configuration(hi,hj,fi,fj,h1,h2,f1,f2):
#     '''A function that tests if arrow i,j,1 and 2 are in same configuration'''
#     return (hi < hj) == (h1 < h2) and (hi < fi) == (h1 < f1) and (hi < fj) == (h1 < f2) and (hj < fi) == (h2 < f1) and (hj < fj) == (h2 < f2) and (fi < fj) == (f1 < f2)

# def configuration_contribution(GD,TC):
#     '''A function that computes all subconfigurations of a TorusGaussCode TC that satifies the condition of 
#     a GaussDiagramWithCoefficient GD that have two arrows'''
#     #Getting the GD condition
#     arrow1 = GD.get_arrowslist()[0]
#     arrow2 = GD.get_arrowslist()[1]
#     hom1 = arrow1.get_homotopy()
#     hom2 = arrow2.get_homotopy()
#     h1 = arrow1.get_head()
#     f1 = arrow1.get_foot()
#     h2 = arrow2.get_head()
#     f2 = arrow2.get_foot()
#     Cont = 0
    
#     #Scanning all arrows of TC
#     for i in range(len(TC.get_Arrows())):
#         arrowi = TC.get_Arrows()[i]
#         homi = arrowi.get_homotopy()
#         wi = arrowi.get_writhe()
#         fi,hi = arrowi.get_coordinates()
#         #Scanning all arrows after arrow i
#         for j in range(i+1,len(TC.get_Arrows())):
#             arrowj = TC.get_Arrows()[j]
#             homj = arrowj.get_homotopy()
#             wj = arrowj.get_writhe()
#             fj,hj = arrowj.get_coordinates()
#             #Checking if the couple satisfies the GD condition
#             if homi == hom1 and homj == hom2:
#                 #Checking for the 4 possible configurations of GD
#                 for k in range(4):
#                     if same_configuration(hi,hj,fi,fj,h1,h2,f1,f2):
#                         Cont = Cont + (wi*wj)
#                     h1 = (h1 - 1)%4
#                     f1 = (f1 - 1)%4
#                     h2 = (h2 - 1)%4
#                     f2 = (f2 - 1)%4
#             elif homj == hom1 and homi == hom2:
#                 #Checking for the 4 possible configurations of GD
#                 for k in range(4):
#                     if same_configuration(hj,hi,fj,fi,h1,h2,f1,f2):
#                         Cont = Cont + (wi*wj)
#                     h1 = (h1 - 1)%4
#                     f1 = (f1 - 1)%4
#                     h2 = (h2 - 1)%4
#                     f2 = (f2 - 1)%4
#     return Cont

# def compute_finite_invariant(I,S):
#     '''A function that compute the value of a Gauss diagram formulae I on a braid data D '''
#     D = BraidData(S)
#     TC = D.get_TorusGaussCode()
#     Inv = 0
#     for GD in I:
#         Inv = Inv + configuration_contribution(GD,TC)*GD.get_coefficient()
#     return Inv

# def print_all_finite_invariants(S):
#     for i in range(len(Tf)):
#         print( 'I%d = '%i, compute_finite_invariant(Tf[i],S))

List of the different Gauss formulas

In [ ]:
# plot_invariant(1,0,0,0,0,0,0,0,'$C_{%d}$'%0,0)
# plot_invariant(0,1,0,0,0,0,0,0,'$C_{%d}$'%1,1)
# plot_invariant(0,0,1,0,0,0,0,0,'$C_{%d}$'%2,2)
# plot_invariant(0,0,0,1,0,0,0,0,'$C_{%d}$'%3,3)
# plot_invariant(0,0,0,0,1,0,0,0,'$C_{%d}$'%4,4)
# plot_invariant(0,0,0,0,0,1,0,0,'$C_{%d}$'%5,5)
# plot_invariant(0,0,0,0,0,0,1,0,'$C_{%d}$'%6,6)
# plot_invariant(0,0,0,0,0,0,0,1,'$C_{%d}$'%7,7)

Looking for a useful example of two braids that don't have the same invariants

In [ ]:
# def generate_all_finite_invariants(S):
#     Lis = []
#     for i in range(len(Tf)):
#         Lis.append(compute_finite_invariant(Tf[i],S))
#     return Lis

# def compute_invariants_cocycle_and_finite_type(S):
#     FInvS = generate_all_finite_invariants(S)
#     InvCocS = List_sigmas(S,T1raf)
#     InvDerivS = []
#     for invariant in InvCocS:
#         if invariant != 0 :
#             InvDerivS.append(invariant.derivative(x0)(1,0,0,0,0,0))
#         else:
#             InvDerivS.append(0)
#     return [FInvS,InvDerivS]

# def recherche_exemple_pertinent(maxi=11,attempt=10,perattempt=20,test=False):
#     i = 1
#     found = False
#     if test:
#         print('DEBUT DE LA RECHERCHE')
#     while i <= attempt and found == False:
#         # Tirage tresse 1
#         if test:
#             print('   Tentative numero {}'.format(i))
#         size = 0
#         while size%2 == 0:
#             size = int(round(random.random()*maxi))
#         S1 = random_braid(size)
#         if test:
#             print('      Tirage de la tresse 1 : {}'.format(S1))
#         [FInvS1,InvDerivS1] = compute_invariants_cocycle_and_finite_type(S1)
#         if test:
#             print('         Invariants de type fini : {}'.format(FInvS1))
#             print('         Invariants de cocycle : {}'.format(InvDerivS1))
#             print('      Tirage de la tresse 2 :')
#         #Tirage tresse 2 
#         j = 1
#         size = 0
#         while size%2 == 0:
#             size = int(round(random.random()*maxi))
#         S2 = random_braid(size)
#         [FInvS2,InvDerivS2] = compute_invariants_cocycle_and_finite_type(S2)
#         if test:
#             print('         {}'.format(S2))
#             print('            {}'.format(FInvS1==FInvS2))
#         while j <= perattempt and FInvS1 != FInvS2:
#             size = 0
#             while size%2 == 0:
#                 size = int(round(random.random()*maxi))
#             S2 = random_braid(size)
#             if test:
#                 print('         {}'.format(S2))
#             [FInvS2,InvDerivS2] = compute_invariants_cocycle_and_finite_type(S2)
#             if test:
#                 print('            {}'.format(FInvS1==FInvS2))
#             j = j + 1
#         #Verification de si on l'a trouvée ou pas
#         if FInvS1 == FInvS2 and InvDerivS1 != InvDerivS2:
#             found = True
#             if test:
#                 print('EXEMPLE TROUVE : {} et {}'.format(S1,S2))
#         i = i + 1
#     return S1,S2,FInvS1,FInvS2,InvDerivS1,InvDerivS2

In [ ]:
# recherche_exemple_pertinent(11,30,10,True)

Example : 

In [ ]:
# S1 = [3, -2, 1, 3, -2, 3, 1]
# S2 = [1, -3, 2]
# [FInvS1,InvDerivS1] = compute_invariants_cocycle_and_finite_type(S1)
# [FInvS2,InvDerivS2] = compute_invariants_cocycle_and_finite_type(S2)
# print('S1 : {}'.format(S1))
# print('S2 : {}'.format(S2))
# print('Invariants de type 2 : ')
# print('   Pour S1 : {}'.format(FInvS1))
# print('   Pour S2 : {}'.format(FInvS2))
# print('Invariants de cocycles combinatoires : ')
# print('   Pour S1 : {}'.format(InvDerivS1))
# print('   Pour S2 : {}'.format(InvDerivS2))

### II.e. Behavior of the invariants with $\Delta^{2n}$

In [ ]:
# def print_results_delta2n(sizemax=7,nmax=4,tries=1):
#     Delta = [1,2,3,1,2,1]
#     Delta2 = Delta+Delta
#     for i in range(tries):
#         size = 0
#         while size%2 == 0 or size < 3:
#             size = int(round(random.random()*sizemax))
#         S = random_braid(size)
#         while S[-1] == -1:
#             S = random_braid(size)
#         n = 0
#         print('For the braid {}'.format(S))
#         Inv = list(List_sigmas(S,T1mult))
#         print('   Multivariable invariants for n = {} : '.format(n))
#         print('      (1,1,-) : {}'.format(Inv[0]))
#         print('      (1,2,-) : {}'.format(Inv[1]))
#         print('      (2,1,-) : {}'.format(Inv[2]))
#         print('      (2,3,+) : {}'.format(Inv[3]))
#         print('      (3,2,+) : {}'.format(Inv[4]))
#         print('      (2,3,+) : {}'.format(Inv[5]))
#         while n < nmax :
#             n = n + 1
#             S = S + Delta2
#             Inv = list(List_sigmas(S,T1mult))
#             print('   Multivariable invariants for n = {} : '.format(n))
#             print('      (1,1,-) : {}'.format(Inv[0]))
#             print('      (1,2,-) : {}'.format(Inv[1]))
#             print('      (2,1,-) : {}'.format(Inv[2]))
#             print('      (2,3,+) : {}'.format(Inv[3]))
#             print('      (3,2,+) : {}'.format(Inv[4]))
#             print('      (2,3,+) : {}'.format(Inv[5]))

In [ ]:
# print_results_delta2n(tries=10)

## III - Clearing all negative crossings (for versions 8.8 and above)
This program doesn't work for Sagemath 8.8 and above if the braid has negative crossings. So we get around that problem using $\Delta^2$. Uncomment if needed

In [ ]:
# def is_positive_braid(S):
#     is_positive = True
#     i = 0
#     while i < len(S) and is_positive == True :
#         s = S[i]
#         if s < 0:
#             is_positive = False
#         i = i + 1 
#     return is_positive

# def push_delta2_positive(S,i):
#     '''a function that pushes Delta2 through S[i]'''
#     #Suppose we have S of the form S'Delta2S''
#     s = S[i]
#     newS = [S[j] for j in range(i)]
#     Delta2 = [1,2,3,1,2,1,1,2,3,1,2,1]
#     newS.extend(Delta2)
#     newS.append(s)
#     newS.extend([S[j] for j in range(i+13,len(S))])
#     return newS

# def clear_negative_crossing(S,i):
#     '''a function that clears a negative crossing S[i] assuming it is followed by Delta2'''
#     s = S[i]
#     newS = [S[j] for j in range(i)]
#     #If s == -1, replace -1123121123121 by 23121123121
#     if s == -1:
#         newS.extend([2,3,1,2,1,1,2,3,1,2,1])
#         newS.extend([S[j] for j in range(i+13,len(S))])
#     elif s == -2:
#         newS.extend([1,2,3,2,1,1,2,3,1,2,1])
#         newS.extend([S[j] for j in range(i+13,len(S))])
#     elif s == -3:
#         newS.extend([1,2,3,1,2,1,2,3,1,2,1])
#         newS.extend([S[j] for j in range(i+13,len(S))])
#     return newS

# def clear_all_negative_crossings(S):
#     '''a function that clears all negative crossings from a braid S'''
#     newS = deepcopy(S)
#     Delta2 = [1,2,3,1,2,1,1,2,3,1,2,1]
#     while is_positive_braid(newS) == False:
#         i = len(newS)-1
#         newS.extend(Delta2)
#         #On pousse Delta2 jusqu'au prochain croisement négatif
#         while newS[i] > 0 :
#             newS = push_delta2_positive(newS,i)
#             i = i - 1
#         #On élimine le croisement négatif
#         newS = clear_negative_crossing(newS,i)
#     return newS

Example

In [ ]:
# S = [-1,2,3,1,2,3,-1,2,-3,2,-1,1,2,1]
# print(clear_all_negative_crossings(S))

In [ ]:
# S = [1,2,3]
# print(clear_all_negative_crossings(S))

# -----------------------------------------------------------------------------
## "User-friendly" interface

Just put your braid after $S = $. The braid has to be of the form $[i]$ where $i$ stands for the crossing $\sigma_i$. 
Then, execute all the next cells : 

In [ ]:
S = 

The knot diagram of the braid in the solid torus : 

In [ ]:
D = BraidData(S)
D.plot_braid_diagram()

The invariants : 

In [ ]:
# Uncomment for n=4 to get the multivariable invariants
# print('Multivariable invariants : {}'.format(list(List_sigmas(S,T1mult)))) 
print('Refined invariants : {}'.format(list(List_sigmas(S,T1raf))))

Example :

In [ ]:
S = [1,2,3,1,2,3,1,2,1,1,2,3,1,2,1]

In [ ]:
D = BraidData(S)
D.plot_braid_diagram()

In [ ]:
# Uncomment for n=4 to get the multivariable invariants
# print('Multivariable invariants : {}'.format(list(List_sigmas(S,T1mult)))) 
print('Refined invariants : {}'.format(list(List_sigmas(S,T1raf))))